In [1]:
import re
import os
import numpy as np
from utils import dict_file
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat

np.set_printoptions(linewidth=1000)

In [ ]:
dim = [5, 9]
for num in range(1, 6):
    plt.figure(figsize=(15, 5))
    for D in dim:
        sub = sorted(os.listdir('./data_322'))[num]
        path = f'./data_322/{sub}'  # path of subfolder
        fidelity = loadmat(f'{path}/fidelity_num{num}_D{D}_L2.mat')['fidelity'][0]
        energy = loadmat(f'{path}/energy_num{num}_D{D}.mat')['energy'][0]
        plt.suptitle(sub, fontsize=14)

        plt.subplot(1, 2, 1)
        plt.xlim(max(energy) + 0.01, min(energy) - 0.01)
        plt.plot(energy, fidelity, label=f'D{D}')
        plt.scatter(energy, fidelity, s=20)
        plt.ylabel('Fidelity', fontsize=12)
        plt.xlabel('Energy', fontsize=12)
        plt.legend(loc='right')

        plt.subplot(1, 2, 2)
        fidelity = 1 - fidelity
        plt.xlim(max(energy) + 0.01, min(energy) - 0.01)
        plt.ylim(max(fidelity) * 2, min(fidelity) / 2)
        plt.plot(energy, fidelity, label=f'D{D}')
        plt.scatter(energy, fidelity, s=20)
        plt.ylabel('1-Fidelity', fontsize=12)
        plt.xlabel('Energy', fontsize=12)
        plt.legend(loc='right')
        plt.yscale('log')
    # plt.savefig(f'./fig/322_{sub}.png')
    plt.show()

In [ ]:
fidelity = {  # D5_L2
    'num1': {
        'vec1': [0.9992741631646074, 0.9993014172110269, 0.998161296040952, 0.9985596117375612, 0.9989809524215505],
        'vec2': [0.9955431697094849, 0.9933422549896271, 0.9922618874777588, 0.9978385677937205, 0.9978066246074941],
        'vec3': [0.9962739068574461, 0.9934214141815548, 0.9863611881600586, 0.9952364837894816, 0.9976951773678556],
        'vec4': [0.9967210935926402, 0.9974019065234625, 0.9933463145445809, 0.9997845104232518, 0.9995254194313064],
        'vec5': [0.9999389065854386, 0.9999194052516596, 0.9998271285730995, 0.9999564644782658, 0.9998395286128368],
        'vec6': [0.9999980693661048, 0.9999861467124065, 0.9999454267673384, 0.7485623326464376, 0.9999713320257472],
        'vec7': [0.9999788493783148, 0.9999841801892319, 0.9999945125537009, 0.9999956106827341, 0.9999887306945812],
        'vec8': [0.9999897836454416, 0.9999985183332956, 0.9999964785688644, 0.9999955498336519, 0.9999960029811199],
        'vec9': [0.9999947000993106, 0.999993127201689, 0.9999960281117449, 0.9999934357491785, 0.9999978087357961],
        'vec10': [0.9999986041039991, 0.9999953648545946, 0.9999988784335423, 0.9999952732680905, 0.9999980839127031],
        'vec11': [0.9999976054297471, 0.9999965959108743, 0.9999963187696214, 0.9999969920334235, 0.9999979993190025],
        'vec12': [0.9999971183859193, 0.999998675833117, 0.999999013041192, 0.9999992459384406, 0.9999980627716535],
        'vec13': [0.9999988439559828, 0.9999999779119939, 0.9999991219032621, 0.9999989978089711, 0.99999866553415],
        'vec14': [0.9999994933814158, 0.9999989182378495, 0.9999995833780693, 0.9999978720293248, 0.9999978610429746],
        'vec15': [0.9999994132308294, 0.9999999999931026, 0.999999239947559, 0.9999990694082331, 0.9999992483130227],
        'vec16': [0.9999993177504714, 0.9999991202640728, 0.9999993107329408, 0.9999781709546003, 0.9999992324443621],
        'vec17': [0.9999977311584773, 0.9999989942090284, 0.9999997160802802, 0.9999999891229281, 0.9999990792057696],
        'vec18': [0.9999986703901871, 0.9999989476374787, 0.9999990607137317, 0.9999994226141622, 0.9993269251128936],
        'vec19': [0.9999996872460526, 0.9999996840252059, 0.9999991326610588, 0.999997954361595, 0.9999993454904116],
        'vec20': [0.999999542874904, 0.9999993830987792, 0.9999998960942609, 0.999999564667966, 0.9999994996175502],
        'vec21': [0.9999990420379729, 0.9999991172338321, 0.9999989519135648, 0.9999998158743658, 0.9999998315780518],
        'vec22': [0.9999996313511446, 0.9999997008401327, 0.999999561573687, 0.9999997576584663, 0.9999993152450862],
        'vec23': [0.9999996217588903, 0.9999996589154173, 0.9999980239164413, 0.9999999264381358, 0.999999506668256],
        'vec24': [0.999999818200055, 0.9999995186487318, 0.9999990414768358, 0.9999986134701588, 0.9999992613283577],
        'vec25': [0.9999998382641356, 0.9999998217862587, 0.9999908222732505, 0.9999998603762962, 0.9999999032926347],
        'vec26': [0.9999997756182493, 0.9999997924175461, 0.9999999055165855, 0.9999995470520541, 0.9999993295771834],
        'vec27': [0.9999991839983557, 0.9999991990274212, 0.9999997589027554, 0.9999989940160493, 0.9999974971224498],
        'vec28': [0.9999969926738566, 0.9999981388055941, 0.9999998771672138, 0.9999995893727299, 0.9999993272357914],
        'vec29': [0.999999243241602, 0.9999993662322735, 0.999999537350746, 0.9999954734999841, 0.9999993957653668],
        'vec30': [0.9999995764641714, 0.9999895656293576, 0.9999997487422791, 0.9999997849812169, 0.9999995433917857],
        'vec31': [0.9999993173925676, 0.9999996245879866, 0.9999999447613357, 0.9999998269131547, 0.9999998847901336],
        'vec32': [0.9999999609620775, 0.9999996685993876, 0.9999985389370659]
    },
    'num2': {
        'vec1': [0.9991780504267896, 0.9988057371945398, 0.9993534069325805, 0.9992105887015786, 0.9995213028399036],
        'vec2': [0.9985015065603607, 0.9992163211457842, 0.995267772477253, 0.9909463489028962, 0.9993726667685476],
        'vec3': [0.9996457215432222, 0.9982002472548149, 0.9997826286691309, 0.9982109189672184, 0.9998061936420385],
        'vec4': [0.9999558044487551, 0.9999866878789693, 0.9999568732650795, 0.999948216686547, 0.9999992413559168],
        'vec5': [0.9999965702559126, 0.999999991437467, 0.9999948804945118, 0.999992701783278, 0.9999912001146296],
        'vec6': [0.9999981278369828, 0.9999999778031871, 0.999996434909606, 0.9999970536532272, 0.9999967046948245],
        'vec7': [0.9999976128495548, 0.9999986279778978, 0.9999960816131577, 0.999999445999441, 0.999999743588722],
        'vec8': [0.9999986543260473, 0.999999132421789, 0.9999995280269534, 0.999996876268921, 0.9999987321468303],
        'vec9': [0.9999986360934171, 0.9999959201736135, 0.9999989966462699, 0.999999581680635, 0.999999389101587],
        'vec10': [0.9999995001055353, 0.9999973239668498, 0.9999988124339376, 0.9999991974656509, 0.9999988487890619],
        'vec11': [0.999999887489827, 0.999999951882557, 0.999999775500124, 0.9999997888990364, 0.9999997228080126],
        'vec12': [0.9999988083191628, 0.9999997721927111, 0.9999994342131651, 0.9999993729167675, 0.9999986641187122],
        'vec13': [0.9999999798765296, 0.9999988471117334, 0.9999990734708673, 0.9999998709450793, 0.9999996258213265],
        'vec14': [0.9999997315233614, 0.9999991799988283, 0.9999993639928628, 0.9999991745638523, 0.9999991597463902],
        'vec15': [0.9999994923484338, 0.9999996478050512, 0.49999991273884914, 0.9999997138061071, 0.9999994160206978],
        'vec16': [0.9999997869341107, 0.9999991413520626, 0.9999977732808589, 0.9999995319481803, 0.9999997697097622],
        'vec17': [0.9999994489819467, 0.9999998208009881, 0.9999984402484839, 0.9999988501425362, 0.9999990422365711],
        'vec18': [0.9999999365041188, 0.9999982363541845, 0.999998854022397, 0.9999988389617023, 0.9999992046516915],
        'vec19': [0.9999996301754115, 0.9999997652166579, 0.9999997956059579, 0.9999983674768665, 0.9999997604988593],
        'vec20': [0.9999987532655945, 0.9999999175449479, 0.9999998582830818, 0.9999993242244141, 0.9999987553124803],
        'vec21': [0.9999997829425671, 0.9999998075557408, 0.9999989494416835, 0.9999997016100044, 0.9999998360480653],
        'vec22': [0.9999995368936901, 0.9999915446931088, 0.9999997419359825, 0.9999992687670689, 0.9999997941904546],
        'vec23': [0.9999999201413471, 0.9999989331368105, 0.9999979952233363, 0.999999997811106, 0.999999809321272],
        'vec24': [0.999999886199274, 0.9999994882762807, 0.9999998432809066, 0.9999983006102202, 0.9999994527536447],
        'vec25': [0.9999996771911372, 0.9999994286725589, 0.9999997452194811, 0.999999334808388, 0.9999997179616247],
        'vec26': [0.9999986450093438, 0.9999990337116839, 0.9999972191836761, 0.9999999066592229, 0.9999993631034652],
        'vec27': [0.999999593515304, 0.9999998999573868, 0.9999999521160619, 0.9999998066580744, 0.9999992734352035],
        'vec28': [0.9999999329056268, 0.9999999624727445, 0.9999998814757266, 0.999999301849122, 0.999998998006355],
        'vec29': [0.9999976627053712, 0.9999999503537518, 0.9999976290100568, 0.9999967550432045, 0.9999998063022901],
        'vec30': [0.9999957630596638, 0.9999998951055378, 0.9999994733355242, 0.49999886999811083, 0.9999999961979616],
        'vec31': [0.9999985498486189, 0.9999994083028766, 0.9999972210258663, 0.9999995653102954, 0.9999960466272413],
        'vec32': [0.9999998284000751, 0.9999996543470606]
    },
    'num3': {
        'vec1': [0.9974755105672704, 0.9966637349948626, 0.9971889394340727, 0.9982500757502197, 0.9976328281798962],
        'vec2': [0.9970957130127999, 0.9965065346807187, 0.995643680857009, 0.9967167325337339, 0.9960554186743931],
        'vec3': [0.9971736107726941, 0.9958391503959094, 0.9973867929045753, 0.9971260115043162, 0.9971652815669964],
        'vec4': [0.9998702310274097, 0.9964225989549477, 0.9986464096502651, 0.9999125439004655, 0.9997923404270032],
        'vec5': [0.9995825502449261, 0.9997659390064791, 0.9997626967983145, 0.9997975214290065, 0.9999808063689689],
        'vec6': [0.9996819096226506, 0.9999634248708982, 0.9998886858947087, 0.9989754016542134, 0.9999998668125099],
        'vec7': [0.9999738431716739, 0.9999787887526936, 0.9999790247980888, 0.9999395872423821, 0.9999734916188809],
        'vec8': [0.9999768102717376, 0.9999904331085363, 0.9999793997628397, 0.9999941398837899, 0.9999930434685396],
        'vec9': [0.9999997708783332, 0.9999975952998152, 0.999993843800253, 0.9999852382321324, 0.9999989527827815],
        'vec10': [0.9999980782043572, 0.9999903047235981, 0.9999901689935671, 0.9999903778838208, 0.9999888031994266],
        'vec11': [0.999996007292064, 0.9999978760974068, 0.9999960618523647, 0.9999980330106462, 0.9999936846519226],
        'vec12': [0.9999985025912292, 0.9999992241133268, 0.99999866566687, 0.9999947489844541, 0.9999961733037863],
        'vec13': [0.9999952578725743, 0.9999961409291014, 0.9999979843527877, 0.9999980689996911, 0.9999980670618753],
        'vec14': [0.9999964896541259, 0.9999986776487967, 0.9999996182894229, 0.9999960199537041, 0.9999973940202914],
        'vec15': [0.9999992791185622, 0.9999977205952353, 0.9999982439387786, 0.999998967643886, 0.9999975893879112],
        'vec16': [0.9999986969454213, 0.9999979507593598, 0.9999981724963007, 0.9999994155419456, 0.999999999992031],
        'vec17': [0.9999981895339383, 0.999998697456745, 0.9999976629149708, 0.9999986853505067, 0.9999982656152048],
        'vec18': [0.9999970111996996, 0.9999985430856646, 0.9999991656193216, 0.9999975574039273, 0.9999985002759517],
        'vec19': [0.9999986093708797, 0.9999979444159155, 0.9999985464899661, 0.9999989041578117, 0.9999989487419209],
        'vec20': [0.9999991154496704, 0.9999997642143093, 0.999999047356624, 0.9999996482069182, 0.9999985205628157],
        'vec21': [0.9999996867321317, 0.9999992519709756, 0.9999992040966472, 0.9999991508293012, 0.9999994837667083],
        'vec22': [0.9999982695156331, 0.9999994511184709, 0.9999993992413179, 0.9999995676231818, 0.9999996413000035],
        'vec23': [0.49999978226172626, 0.9999994818702358, 0.9999974244159768, 0.999999101861221, 0.9999995788131981],
        'vec24': [0.9999990373668102, 0.9999981917112553, 0.9999998502510022, 0.999999531236677, 0.9999996693040054],
        'vec25': [0.9999989030404624, 0.9999984087433138, 0.9999992348924084, 0.9999969494648642, 0.9999960548012128],
        'vec26': [0.9999995259812446, 0.9999940837245769, 0.9999984778374822, 0.9999995379446518, 0.9999995767481428],
        'vec27': [0.9999996138116362, 0.9999988101555504, 0.9999990595840078, 0.999999416215896, 0.9999999441745686],
        'vec28': [0.9999996643033069, 0.9999973876144302, 0.9999993636250994, 0.9999981354488541, 0.9999966563291283],
        'vec29': [0.9999995823163854, 0.9999995223951514, 0.9999998267253714, 0.9999983080793602, 0.9999992236635611],
        'vec30': [0.9999998396091871, 0.999999356814346, 0.999999748253113, 0.9999996423708924, 0.9999994324605488],
        'vec31': [0.999999672645886, 0.9999992748723019, 0.9999999010330961, 0.9999998655769144, 0.9999993329375811],
        'vec32': [0.9999997471465962, 0.9999995804360952, 0.9999996247316321, 0.9999993466733381, 0.9999979331418591],
        'vec33': [0.9999997209535741, 0.9999983326349541, 0.9999990460385411, 0.9999982030364327, 0.9999982961229412],
        'vec34': [0.9999998396171978, 0.9999995309793988, 0.9999995046440115, 0.9999996990024165, 0.999999394436132],
        'vec35': [0.9999999388987014, 0.9999996831219606, 0.9999995949497146, 0.9999990918714705, 0.9999996733911867],
        'vec36': [0.999999533395852, 0.9999999074376493, 0.9999996987691729, 0.999999915138619, 0.9999996132591782],
        'vec37': [0.9999979216466572, 0.7474276245307113, 0.9999996848423296, 0.9999978677274273, 0.9999994079332867],
        'vec38': [0.9999996717554818, 0.9999994384288662, 0.999999043218833, 0.9999990747227467]
    },
    'num4': {
        'vec1': [0.9969906434194008, 0.9944427301060443, 0.9952445886766289, 0.9940521807812964, 0.9975055317918352],
        'vec2': [0.9908453693185165, 0.9860228777996947, 0.9919019430907227, 0.9866733718144363, 0.9906291827094346],
        'vec3': [0.9974233374124295, 0.9981472067970589, 0.998369007133225, 0.9977924922034285, 0.9985600154296536],
        'vec4': [0.9986145727437792, 0.9984396976299552, 0.9993225770122252, 0.9861556427299858, 0.9882999246485124],
        'vec5': [0.9982467098816279, 0.9984244952964746, 0.9991918003692238, 0.9986735379475179, 0.9995421914642159],
        'vec6': [0.9999235080022921, 0.999991161920416, 0.9999983776471515, 0.9999772187683855, 0.9996612335044671],
        'vec7': [0.9999747478904961, 0.9999953020161556, 0.999933135166696, 0.9999100287734162, 0.9999191062516538],
        'vec8': [0.9999614652695029, 0.9999393348287137, 0.9999401955528003, 0.9999503596070345, 0.9999827614245353],
        'vec9': [0.9999761481337512, 0.9999633565904763, 0.9999802241454776, 0.9999676207477208, 0.9999794517487433],
        'vec10': [0.9999806789550127, 0.9999822638215315, 0.9999814491055957, 0.9999942986243822, 0.9999777373347738],
        'vec11': [0.9999877293596758, 0.9999866639424647, 0.9999837682916344, 0.9999861682158604, 0.9295439656712956],
        'vec12': [0.99998899846167, 0.9999914348542421, 0.9999927238640606, 0.9999898923906267, 0.999991108963546],
        'vec13': [0.9999932545165118, 0.9999975971878392, 0.9999988166523205, 0.9999916195793086, 0.9999894939591423],
        'vec14': [0.9999973406978493, 0.9999941706473514, 0.9999952400511695, 0.9999959672209646, 0.9999999633053372],
        'vec15': [0.9999974170121871, 0.9999991637063598, 0.9999951640313758, 0.9999950332315347, 0.9999980416455754],
        'vec16': [0.9999962107395625, 0.9999998662191273, 0.9999966729496597, 0.9999971791537665, 0.9999972502879438],
        'vec17': [0.9999974622579547, 0.9999963826661593, 0.9999974654858349, 0.9999975818105109, 0.9999964755072932],
        'vec18': [0.999996046809382, 0.999999138451641, 0.9999955699015207, 0.9999985994695535, 0.9999994709111999],
        'vec19': [0.9999975505333765, 0.9999977680164236, 0.9999988285176727, 0.9999951156387537, 0.9999983302690639],
        'vec20': [0.9999985595301925, 0.9999976822803993, 0.9999984274202111, 0.9999998636450695, 0.9990288078858742],
        'vec21': [0.9999989845583075, 0.9999953905805294, 0.9999986982777601, 0.9999959532092765, 0.9999977643450826],
        'vec22': [0.9999991664454744, 0.9999988887268412, 0.9999986039217631, 0.9999988080390512, 0.9999954315162325],
        'vec23': [0.999998689258785, 0.9999985402697007, 0.9999998111223584, 0.9999990278678477, 0.9999996601602739],
        'vec24': [0.9999998331603135, 0.9999999485222741, 0.4999996461047139, 0.9999996128219352, 0.9999992052870866],
        'vec25': [0.9999991506842215, 0.9999990023957821, 0.9999991787551605, 0.9999992455973223, 0.9999990711819841],
        'vec26': [0.9999991016548199, 0.9999992384750578, 0.9999998487204287, 0.9999991878461453, 0.9999988016644872],
        'vec27': [0.9999996104862742, 0.9999987260492369, 0.9999994153036365, 0.9999993922540057, 0.999998958338828],
        'vec28': [0.9999993358027792, 0.9999994513487425, 0.9999991078216339, 0.9999998318887322, 0.9999986041173352],
        'vec29': [0.9999999999991195, 0.9999993986774429, 0.9999994173048172, 0.9999992878972392, 0.9999993632127822],
        'vec30': [0.9999992012217155, 0.9999990971978245, 0.9999994861182073, 0.9999994399445448, 0.9999993434866687],
        'vec31': [0.999999282033144, 0.9999984389379768, 0.9999995905531507, 0.9999997515793281, 0.9999991981461678],
        'vec32': [0.9999994972019529, 0.9999958275819745, 0.9999986911996587, 0.9999994843962837, 0.9999457072544349],
        'vec33': [0.9999995744182826, 0.9999998163201933, 0.9999989728254829, 0.9999992116354962, 0.9999999937588997],
        'vec34': [0.9999993615461571, 0.9999947049139181, 0.9999995074375244, 0.9999998537499971, 0.9999922380805493],
        'vec35': [0.999999322318716, 0.999999398334094, 0.9999987824001159]
    },
    'num5': {
        'vec1': [0.9984858956327886, 0.9963606335573533, 0.9957571903115661, 0.9939211822289411, 0.9947933039281487],
        'vec2': [0.9995207159554849, 0.9986025320019913, 0.9993800201751111, 0.9990310926686017, 0.9979847676306459],
        'vec3': [0.9997213031223166, 0.9992054294638983, 0.9994416924931048, 0.999503285200919, 0.9998428071249669],
        'vec4': [0.9999441395397394, 0.9999151783133338, 0.9999313274121417, 0.9999054075217451, 0.9996042335537438],
        'vec5': [0.9999595126066759, 0.9999763852450492, 0.9999547247218745, 0.9999222468636582, 0.9999805809972252],
        'vec6': [0.9999912772054266, 0.9999937235237965, 0.9999894262245297, 0.9999887705451018, 0.999989744047059],
        'vec7': [0.9999944809306914, 0.9999939595946737, 0.9999923278365538, 0.9999922062116731, 0.9999872473793061],
        'vec8': [0.9999582821380923, 0.9999953763941388, 0.999996904174527, 0.9999933266663883, 0.9999957781184174],
        'vec9': [0.9999977734620207, 0.999998447983135, 0.9999984549986044, 0.9999976287295012, 0.9999967579509742],
        'vec10': [0.9999970376193383, 0.9999984066855517, 0.9999970238381426, 0.9999974552555713, 0.9999977975980815],
        'vec11': [0.9999988042424335, 0.9999990251414559, 0.9999992106383077, 0.9999986999190527, 0.9999990200460201],
        'vec12': [0.9999993706942516, 0.9999990485938733, 0.9999978967031451, 0.9999984110032297, 0.9999990222392473],
        'vec13': [0.9999991874955343, 0.9999994804280249, 0.9999993866107105, 0.9999995325344029, 0.9999990595723185],
        'vec14': [0.9999997244960191, 0.9999997133379491, 0.9999997178125548, 0.9999989811837214, 0.9999981230115924],
        'vec15': [0.999999455024417, 0.9999994564493073, 0.9999998466871616, 0.9999995741249941, 0.9999996295268878],
        'vec16': [0.9999954470184764, 0.9999990554557869, 0.9999989697597107, 0.9999987738901621, 0.999997399002057],
        'vec17': [0.9999998438262974, 0.9999996712723518, 0.9999996927750432, 0.9999997208175962, 0.9999998145689896],
        'vec18': [0.9999998749585881, 0.9999993087795205, 0.9999997353845581, 0.9999994612975995, 0.9999996193420932],
        'vec19': [0.9999996042948962, 0.9999941251718029, 0.9999998437241118, 0.9999991471021036, 0.9999996439376415],
        'vec20': [0.9999977287308974, 0.9999984665718793, 0.9999998753303132, 0.9999985511600026, 0.999997522139226],
        'vec21': [0.9999999501277963, 0.9999997618526676, 0.9999990658705907, 0.9999997728284119, 0.9999997735763901],
        'vec22': [0.999999372400663, 0.9999987011349272, 0.999999658299989, 0.9999996047192993, 0.9999998823542029],
        'vec23': [0.9999992933242401, 0.9999998327956442, 0.9999999668595881, 0.9999999448713722, 0.9999989680981898],
        'vec24': [0.999999528985722, 0.9999998243076871, 0.4999999653022239, 0.9999993294505671, 0.9999999419363073],
        'vec25': [0.9999997662991383, 0.9999997984318892, 0.9999997840530701, 0.9999998802503834, 0.9999999373482931],
        'vec26': [0.9999980514036912, 0.9999998243418212, 0.999999965996358, 0.9999999516539589, 0.9999995488403293],
        'vec27': [0.9999999823727177, 0.9999835330209758, 0.9999997294972892, 0.9999999431216735, 0.9999983780358059],
        'vec28': [0.9999994211250849, 0.999999701671922, 0.9999998369577181, 0.9999988333483033, 0.999999969907523],
        'vec29': [0.999999661331716, 0.9999995250877769, 0.9999973862462768, 0.99999997357639, 0.99999952017095],
        'vec30': [0.9999971307468272, 0.9999995769408184, 0.9999998958144904, 0.49999941844863705, 0.9999994936981644],
        'vec31': [0.9999999703744813, 0.999997757480171, 0.9999991258111116, 0.9999994925727118, 0.9999944939348983],
        'vec32': [0.9999995917392608, 0.999999100455323]
    }
}
for num in range(1, 6):
    sub = sorted(os.listdir('./data_322'))[num]
    path = f'./data_322/{sub}'  # path of subfolder
    # print(num, fidelity[f'num{num}'])
    # savemat(f'{path}/fidelity_num{num}_D5_L2.mat', fidelity[f'num{num}'])

In [ ]:
fidelity = {  # D6_L2
    'num1': {
        'vec1': [0.988396215729429, 0.9924862299365363, 0.992812863965521, 0.9940983903265397, 0.9908821439788924],
        'vec2': [0.9901479938915028, 0.9901220007582362, 0.9905482379666798, 0.9899515126779608, 0.992787440379311],
        'vec3': [0.985513715588386, 0.9860042083491276, 0.9937188263597441, 0.9887576778729519, 0.9730010859899564],
        'vec4': [0.9991934310401368, 0.9999941422014987, 0.9999868297517017, 0.9999918932377277, 0.9997107120507656],
        'vec5': [0.9999988926983079, 0.9998234208642066, 0.9999988026010627, 0.9998557560467847, 0.9998074113463493],
        'vec6': [0.9999325685544825, 0.999957362123711, 0.999995935211955, 0.9999873645215227, 0.9999712481694207],
        'vec7': [0.9999910407854993, 0.9999802313245827, 0.9999930283441036, 0.9999791910650212, 0.9999768843469687],
        'vec8': [0.999990073400813, 0.9999955165914739, 0.9999966922268122, 0.9999883362947294, 0.9999746973237913],
        'vec9': [0.9999907043265635, 0.9999930097735734, 0.9999915918009203, 0.9999942614867448, 0.9999948845832745],
        'vec10': [0.9999925318426293, 0.9999936531565158, 0.9999964411928908, 0.9999975213249772, 0.9999996106839592],
        'vec11': [0.9999969796356185, 0.9999976786319628, 0.999980589132357, 0.9999967307417923, 0.9999988437832488],
        'vec12': [0.9999975187582649, 0.9999999407364359, 0.9999999620344419, 0.9998875527672705, 0.9999984608944683],
        'vec13': [0.9999978438980561, 0.9999992221877885, 0.9999982561464245, 0.9999990669640197, 0.9999977945244515],
        'vec14': [0.9999987141585789, 0.9999990495918238, 0.9999997297622656, 0.99999849365056, 0.9999987255613204],
        'vec15': [0.9999994250943578, 0.9999995366805379, 0.9999991510222692, 0.9999945615669641, 0.9999991910360204],
        'vec16': [0.9999992841747443, 0.9999997012017563, 0.999999345631021, 0.9999995474273206, 0.9999996544737569],
        'vec17': [0.9999986570238381, 0.9999993018050619, 0.9999993271868085, 0.9999990922151907, 0.9999997734385926],
        'vec18': [0.9999994918050198, 0.9999995965712958, 0.9999996861919164, 0.9999994324455919, 0.999999999050178],
        'vec19': [0.9999990297880761, 0.9999995952115636, 0.9999997969481007, 0.9999997928687703, 0.9999995884004619],
        'vec20': [0.9999985148777155, 0.9999993259374451, 0.9999993368736293, 0.9999985812677862, 0.9999995867827983],
        'vec21': [0.999990376199175, 0.9999996829763808, 0.9999980061500074, 0.9999993362550602, 0.9999999106865408],
        'vec22': [0.9999967291496922, 0.9999996286571072, 0.9999995611022464, 0.9999998136238712, 0.9999996414988743],
        'vec23': [0.999999294438102, 0.9999999747691515, 0.9999990683926644, 0.9999983384999838, 0.9999999954866317],
        'vec24': [0.9999997270646226, 0.9999836257260042, 0.9999998293246295, 0.9999976747109813, 0.9999997576348083],
        'vec25': [0.9999990786145656, 0.9999998537529533, 0.9999995167476217, 0.9999993202572248, 0.9999999303917935],
        'vec26': [0.9999997649414003, 0.9999994961357792, 0.9999996705796451, 0.999999631318713, 0.9999997472512174],
        'vec27': [0.9999998461634033, 0.999995512189006, 0.9999992888401711, 0.999999809970423, 0.9999998608440046],
        'vec28': [0.9999985015738554, 0.9999991479746367, 0.9999985676917567, 0.9999994274892487, 0.9999995771155195],
        'vec29': [0.9999983429815968, 0.9999997646195776, 0.9999998698345847, 0.9999975351145826, 0.9999988936773769],
        'vec30': [0.9999998799858962, 0.9999998476060208, 0.9999999196648095, 0.999999874525576, 0.9999991797555711],
        'vec31': [0.999996876407542, 0.9999941423947509, 0.9999985424702788, 0.9999999553579026, 0.9999984478876099],
        'vec32': [0.9999997700589973, 0.9999998996553883, 0.9999989247269444, 0.9999995764298699, 0.9999998509274004],
        'vec33': [0.99999995410277, 0.9999972508468636, 0.9999970976082797, 0.9999998247510915, 0.9999997466614499],
        'vec34': [0.9999990597864448, 0.9999997851483063, 0.9999993737753892, 0.999998673845953, 0.999999919688764],
        'vec35': [0.9999994198783257, 0.9999993817980319, 0.9999997086663641, 0.999999823421213, 0.999999800829262],
        'vec36': [0.9999999698699766, 0.9998823071397513, 0.9999997839587795, 0.9999996284199915, 0.9999996997342007],
        'vec37': [0.9999992328396134, 0.9999998142245226, 0.9999995661077146, 0.9999999579588157, 0.999999265651911],
        'vec38': [0.9999999290825726, 0.9999895288657744, 0.9999997569316807, 0.9999998073399725, 0.9999999326196496],
        'vec39': [0.9999961489726591, 0.9999998189396094, 0.9999977283400058, 0.9042316805337717, 0.9999994853078408],
        'vec40': [0.9999993869799316, 0.9999945554392843, 0.9999999205206429, 0.9999988039943822, 0.9999992166472607]
    },
    'num2': {
        'vec1': [0.9975236899372091, 0.9969111336774399, 0.9974612293016697, 0.9964737568419102, 0.9971089284254578],
        'vec2': [0.9989019636026198, 0.9989316476230374, 0.9989388168985401, 0.9989673116488207, 0.9986239546054163],
        'vec3': [0.9988801466549247, 0.9988518965016552, 0.999083356168095, 0.9990213526659072, 0.9985984994961856],
        'vec4': [0.9999623558412699, 0.9999551941004693, 0.9999881759316418, 0.999950199835147, 0.9999992217248858],
        'vec5': [0.9999944128805678, 0.5000001518491679, 0.9999923953081848, 0.9999917782164186, 0.9999965434866687],
        'vec6': [0.9999968725822657, 0.9999961796665369, 0.9999960565074769, 0.9999952399835011, 0.9999909654472392],
        'vec7': [0.9999978392516737, 0.9999992963261701, 0.9999988403729134, 0.9999984497866232, 0.9999995454225961],
        'vec8': [0.9999990928460908, 0.9999928519875879, 0.9999998463701694, 0.9999986562572383, 0.999998232700965],
        'vec9': [0.9999991021810928, 0.9999988588061476, 0.9999992741880342, 0.9999991284284286, 0.9999980614372156],
        'vec10': [0.9999992172672247, 0.999990556608382, 0.9999982427468299, 0.9999995247678366, 0.9999998926035716],
        'vec11': [0.9999991398295038, 0.9999999419666381, 0.9999993118478568, 0.9999989937573686, 0.9999991130963299],
        'vec12': [0.9999996911218427, 0.9999992434251346, 0.9999997177605967, 0.9999991618181837, 0.9999999673920472],
        'vec13': [0.9999997432609606, 0.9999991510977724, 0.9999993923682915, 0.9999995971341062, 0.9999998858771595],
        'vec14': [0.9999992356430669, 0.9999998290087107, 0.9999968488763235, 0.9999995481266486, 0.9999996460459077],
        'vec15': [0.9999995614803343, 0.9999967201887491, 0.9999994017949703, 0.9999993487914998, 0.9999995657608269],
        'vec16': [0.9999997646369616, 0.9999996759300223, 0.9999998419735017, 0.9999990369082383, 0.9999992682822394],
        'vec17': [0.9999985465649933, 0.9999995264990027, 0.9999995492526005, 0.9999989898115261, 0.9999995031616241],
        'vec18': [0.9999992053493969, 0.9999994521498663, 0.9999996735637955, 0.999999644333387, 0.999999994810437],
        'vec19': [0.9999996728679823, 0.9999999272008092, 0.9999951078363807, 0.9999982361053824, 0.9999980950253093],
        'vec20': [0.999998886031845, 0.9999995094776917, 0.9946784890235465, 0.999999742555945, 0.9999987537271613],
        'vec21': [0.9999998277003479, 0.9999997942156582, 0.9999998385484388, 0.9999999400028583, 0.9999959959215589],
        'vec22': [0.9999997518640953, 0.9999999431352123, 0.9999997569463739, 0.9999954931670816, 0.9999998302927076],
        'vec23': [0.9999996902090809, 0.9999992192699934, 0.9999995743515279, 0.9999631997041992, 0.999999665880565],
        'vec24': [0.9999991792573861, 0.9999999638376935, 0.9999997168818902, 0.99999940663504, 0.999999925112058],
        'vec25': [0.9999999273987747, 0.9999998892252553, 0.9999953573254479, 0.9999999519165579, 0.9999998649728041],
        'vec26': [0.999999882919185, 0.9999999401550176, 0.999999825377015, 0.9999999237507441, 0.9999994109040365],
        'vec27': [0.9999999031688752, 0.9999976905456438, 0.9999998618974365, 0.9999999689769778, 0.9999997410569181],
        'vec28': [0.9999974347812068, 0.9999998914496162, 0.9999994079802672, 0.9999985554333797, 0.9999993133957146],
        'vec29': [0.9999997192439405, 0.9999977826480076, 0.9999994758780001, 0.9999938135215424, 0.9999988851988015],
        'vec30': [0.9999998415602227, 0.9999997776045791, 0.9999982587775742, 0.9999998297439521, 0.9999996153873404],
        'vec31': [0.9999998821784747, 0.9999975940147777, 0.9999995919902855, 0.9999996996761928, 0.9999994498292284],
        'vec32': [0.9999995520244382, 0.9999992288042248, 0.9999998827307137, 0.9999992850973427, 0.9999997721123333],
        'vec33': [0.9999999477699282, 0.9999985268416312, 0.9999994611200067, 0.9999986555083893, 0.9999991924987117],
        'vec34': [0.9999993842047571, 0.9999913496050916, 0.9999998156873778, 0.9999987970728397, 0.9999996064361586],
        'vec35': [0.9999990256134812, 0.99999997346782, 0.9999989278297439, 0.999996704303689, 0.9999985493810398],
        'vec36': [0.9999998231274794, 0.9999994126458022, 0.9999999039368996, 0.9999996371131707, 0.9999992847984165],
        'vec37': [0.9999988160058004, 0.9999989402170708, 0.9999996132849691, 0.9999996972253621, 0.9999997146165722],
        'vec38': [0.9999995650802358, 0.9999997026283818, 0.9999993109688408, 0.9999997903308032, 0.9999999284327574],
        'vec39': [0.9999998671212461, 0.9999999506683933, 0.999999315993225, 0.9999999153348094, 0.9999999732178501],
        'vec40': [0.9999981903623789, 0.9999992769197503, 0.999999844923384, 0.9999987543195689, 0.9999995393498188]
    },
    'num3': {
        'vec1': [0.9761891358594893, 0.9753406639260325, 0.9793835315596354, 0.9722862223026884, 0.979503328544081],
        'vec2': [0.9928488414371007, 0.9956118614069434, 0.993800294070881, 0.9933774704415452, 0.9914350993875979],
        'vec3': [0.9709387226316346, 0.9702160008583525, 0.9704118951839344, 0.9689951170637541, 0.9686035340386974],
        'vec4': [0.9904399732639713, 0.994754365930357, 0.9960618171863223, 0.9962350448207313, 0.9892572805329072],
        'vec5': [0.9999856056899851, 0.9988872600308468, 0.9997000637902258, 0.9999258308092466, 0.9998496298310771],
        'vec6': [0.99886426326139, 0.9987544737298628, 0.9978400865607127, 0.7260346792821996, 0.9988944400706109],
        'vec7': [0.9999890412112231, 0.9999850290902893, 0.9999626765707599, 0.9999696701301192, 0.9999828404092469],
        'vec8': [0.999984345858636, 0.9999917440526405, 0.9999787644362195, 0.8749913012557098, 0.9999983922678859],
        'vec9': [0.9999909185865755, 0.9999889012339804, 0.9999995779763945, 0.999994033174862, 0.999986470293976],
        'vec10': [0.99999016031738, 0.9999968794590339, 0.999998961383793, 0.49999816758043236, 0.9999940277334782],
        'vec11': [0.9999978449289219, 0.999992767967293, 0.9999929610804393, 0.9999973147765867, 0.9999965425405866],
        'vec12': [0.9999972037108444, 0.9999952021128953, 0.9999973429224784, 0.9999975959824962, 0.9999976573936235],
        'vec13': [0.999997377292304, 0.9999982125556979, 0.9999999795252531, 0.9999966694382234, 0.9999962198033288],
        'vec14': [0.9999984310284421, 0.9999973248746282, 0.9999247916529599, 0.9999973109323671, 0.9999989294546621],
        'vec15': [0.9999980102676254, 0.9999987885317573, 0.9999975702294357, 0.9999980033385053, 0.9999946393139035],
        'vec16': [0.999998766668756, 0.9999876905498751, 0.9999986736917901, 0.9999982650721588, 0.9999981835821128],
        'vec17': [0.9999981201552686, 0.9999981078326952, 0.9999984985940852, 0.999999177905904, 0.9999996593501822],
        'vec18': [0.9999989857114383, 0.9999992796400126, 0.9999990684412539, 0.9999971683317312, 0.9999994405097605],
        'vec19': [0.9999992757047366, 0.9999999993711413, 0.9999827461693785, 0.999999107344601, 0.9999972777377428],
        'vec20': [0.9999996702232713, 0.9999960489622113, 0.9999990215326363, 0.9999992368264768, 0.9999998347808763],
        'vec21': [0.999999421864678, 0.9999994637920089, 0.9999989054502808, 0.9999987247246129, 0.9999994323729833],
        'vec22': [0.9999902392331073, 0.9999998465648583, 0.9999958710754201, 0.9999996969699314, 0.9999996464912383],
        'vec23': [0.9999886792526838, 0.9999977791675043, 0.9999889366966828, 0.9999989838674312, 0.9999988659740373],
        'vec24': [0.9999991101448653, 0.9999997450247724, 0.9999996569846566, 0.9999993617277367, 0.9999997885019746],
        'vec25': [0.9999997345626975, 0.9999970068794749, 0.9999993561178514, 0.9999997654163497, 0.9999830531437763],
        'vec26': [0.999999709916885, 0.9999952992948128, 0.9999996287070146, 0.9999997446354006, 0.9999933317670617],
        'vec27': [0.9999995342351721, 0.9999996609691177, 0.9999982725896632, 0.9999994829254946, 0.9999990333408331],
        'vec28': [0.9999995876321316, 0.9999992396818245, 0.9999998116173439, 0.9999986389599238, 0.9999998797892503],
        'vec29': [0.9999993871746713, 0.9999996259191226, 0.9999905740804873, 0.9999950361471063, 0.9999993064029206],
        'vec30': [0.9999997058755621, 0.9999988487756626, 0.9999997040525277, 0.9999996510685126, 0.9999993473419913],
        'vec31': [0.9999972614547039, 0.9999993696413132, 0.9999995535190387, 0.9999988455887105, 0.999999776373268],
        'vec32': [0.9999994689933097, 0.9999997577967836, 0.9999990817849491, 0.9844171868511817, 0.9999996407813622],
        'vec33': [0.9999970985259559, 0.9999991466426761, 0.9999994249839708, 0.9999997948298149, 0.9999997176354148],
        'vec34': [0.9999985158104934, 0.9999988631153078, 0.9999996628228826, 0.9999998830344325, 0.9999993912107938],
        'vec35': [0.9999994439844977, 0.999999844413254, 0.9999992602488241, 0.9999922716488395, 0.9999988480376558],
        'vec36': [0.9999989686594283, 0.9999992934023814, 0.9999996768488693, 0.9999995611051622, 0.9999993567040617],
        'vec37': [0.9999996829960219, 0.9999995394551575, 0.9999992208152904, 0.9999986748024141, 0.9999997725941971],
        'vec38': [0.9999997380949115, 0.999930538607352, 0.999998446823793, 0.9999996059994976, 0.9999990391609497],
        'vec39': [0.9999995101888224, 0.9999833991662449, 0.9774811477675822, 0.9999974987980662, 0.9999997882511933],
        'vec40': [0.9999975043104141, 0.9999998635967688, 0.9999948637080844, 0.9999997949792083, 0.9999996643590341]
    },
    'num4': {
        'vec1': [0.9758438197899528, 0.9720003244944531, 0.9677710096310437, 0.9739300569628675, 0.9769193349205898],
        'vec2': [0.9944242710979672, 0.9950735031389855, 0.9944203272448674, 0.9949592658118037, 0.9947869576212526],
        'vec3': [0.99761795944395, 0.9975591743726459, 0.9976382178852813, 0.9981187104858502, 0.9981502637923009],
        'vec4': [0.9789399401535315, 0.9895346905571172, 0.9908978484489626, 0.9908647414372707, 0.9903671285876205],
        'vec5': [0.9999515737601872, 0.9999681301610606, 0.9404127755709605, 0.9996612194246157, 0.9999666116179935],
        'vec6': [0.9997161093961469, 0.9994936103815036, 0.9998857881405787, 0.9998134984177656, 0.9998301325030726],
        'vec7': [0.9999132072633924, 0.9999438793822231, 0.9998382898093713, 0.9998665521392028, 0.9998624915558381],
        'vec8': [0.9301917482998883, 0.999982502184916, 0.9999977348499623, 0.999958415466412, 0.9999839892273561],
        'vec9': [0.9300145266323854, 0.9999710213869653, 0.9999946613871142, 0.9999824009572681, 0.9999908648892534],
        'vec10': [0.9999959936820156, 0.9999700635226888, 0.9999999429674383, 0.9999924748952873, 0.9999784935716037],
        'vec11': [0.9999858804914379, 0.9999981433116945, 0.9999939507500839, 0.9999974724628735, 0.9999956034245456],
        'vec12': [0.9999989904029863, 0.9999906130217187, 0.9999884172381618, 0.9999993975647125, 0.9999953897093238],
        'vec13': [0.999991543772595, 0.9974573579085381, 0.9999912153965922, 0.998792549215421, 0.9999995909785668],
        'vec14': [0.9999950451399364, 0.9999987012153875, 0.9999998927748444, 0.9999934504772245, 0.9999924909365167],
        'vec15': [0.9999966969190976, 0.9851534875287311, 0.9999933555433312, 0.9999954293330685, 0.9999980315261807],
        'vec16': [0.99999861407774, 0.9999984870666109, 0.9999960631384335, 0.9999955201700668, 0.9999991200487068],
        'vec17': [0.9999995360904514, 0.9999987119935156, 0.9999988180342152, 0.999997194613736, 0.9999971016194759],
        'vec18': [0.9999990246070308, 0.9953794617585219, 0.9999977621614726, 0.9999975473489648, 0.9999978695478768],
        'vec19': [0.9999982536900967, 0.9999985674503211, 0.999998789565032, 0.9999990554356992, 0.9999956666483651],
        'vec20': [0.999998716269727, 0.9999994933944336, 0.9999999959321598, 0.9999988090934064, 0.9999991174377839],
        'vec21': [0.9999995996193647, 0.9999993501733133, 0.9999980205408996, 0.9999995364667695, 0.9999995654896705],
        'vec22': [0.9999992409881133, 0.9999986298829269, 0.9999980167302923, 0.9999984394720156, 0.9999973751441502],
        'vec23': [0.9999988888885845, 0.9999989677554536, 0.9999988681073261, 0.9999991694421003, 0.999999082176296],
        'vec24': [0.9999995126930985, 0.9999998449223546, 0.9999987313351103, 0.9999990555137666, 0.9999986715946199],
        'vec25': [0.999998771444665, 0.9999990917452681, 0.9999997408448528, 0.9999990763059763, 0.9999987738122835],
        'vec26': [0.9999991829656643, 0.9999993265975331, 0.9999986821763281, 0.9999993715094847, 0.999998871888702],
        'vec27': [0.9999993377024889, 0.999998334449756, 0.9999992387775822, 0.9999997312539948, 0.9999990839033643],
        'vec28': [0.9999991456375541, 0.999999467422877, 0.999998727087993, 0.9999999654174367, 0.9999993864759624],
        'vec29': [0.9999992554586329, 0.9999996563385207, 0.9999993405956527, 0.9999988155044928, 0.9999997192107158],
        'vec30': [0.9999994451510983, 0.9999990092240454, 0.9999997983454434, 0.9999996786931103, 0.9999987833371599],
        'vec31': [0.9999994370334548, 0.9215458816428099, 0.9999991895516754, 0.9999994812288197, 0.9999986856655524],
        'vec32': [0.9999693762910041, 0.9999995232680352, 0.9999995179103678, 0.999999386829511, 0.9999990741765548],
        'vec33': [0.99999914247675, 0.9205531000938506, 0.9999995105063025, 0.9999994308112216, 0.9999995384747182],
        'vec34': [0.9999997311080838, 0.9999994527657613, 0.999999755114371, 0.9999869256346723, 0.999999855548334],
        'vec35': [0.999999554221759, 0.9999995634110707, 0.9999993672181031, 0.9999986971202872, 0.9999958850106512],
        'vec36': [0.9999998771270815, 0.9999996273790696, 0.9999999842713566, 0.999999740206085, 0.9999996918076786],
        'vec37': [0.9999995889971438, 0.9999993395445329, 0.999998632028833, 0.9999992068615291, 0.9999992550732664],
        'vec38': [0.9999313565868262, 0.9999996328718329, 0.9999990232247102, 0.9999994350645546, 0.9999994829712323],
        'vec39': [0.9999995368545999, 0.9999994720760178, 0.9999999897142281, 0.9999996972330373, 0.9999997057164591],
        'vec40': [0.9999996686559037, 0.9999994945985313, 0.9999995585385744, 0.9999990737357487, 0.9999993112072122]
    },
    'num5': {
        'vec1': [0.9955016551081799, 0.9958335898114831, 0.9939926414120062, 0.9924854729588777, 0.9950895702366234],
        'vec2': [0.998165169396897, 0.9945749919306335, 0.9933821718810725, 0.9969337738848445, 0.9976231504918265],
        'vec3': [0.9992835608276718, 0.9996934859689897, 0.9996732658907745, 0.9992201140150936, 0.9948218949153596],
        'vec4': [0.9999184370867842, 0.9999070307313179, 0.999915670040017, 0.9998836026915672, 0.9999271614294114],
        'vec5': [0.9999603029322038, 0.9999700761360464, 0.9999722950820282, 0.9999807552234081, 0.9999854335905383],
        'vec6': [0.9999882234186838, 0.9999924782389404, 0.9999832985073657, 0.9999904662641518, 0.9999824217120797],
        'vec7': [0.9999929831176944, 0.9999803880638876, 0.9999943217292747, 0.9998065812793989, 0.9999948538827088],
        'vec8': [0.9999929306360351, 0.9999954696477769, 0.999996030245426, 0.9997911357382969, 0.9807229543562519],
        'vec9': [0.9999971503433931, 0.9999966422664761, 0.9999989216791375, 0.9999975447961328, 0.9999981081414505],
        'vec10': [0.9999988098952997, 0.9999970151297911, 0.9999984801520193, 0.9999984073907401, 0.9999986147583215],
        'vec11': [0.9999982500207442, 0.9999986011752583, 0.9999702705162359, 0.9999986404368045, 0.9999984571971324],
        'vec12': [0.999998523015044, 0.9999984132002787, 0.9999989723974805, 0.9999974690026906, 0.9999984596659324],
        'vec13': [0.9999994887114496, 0.9999993234982927, 0.9999984877934234, 0.9999992534749149, 0.9999993830750311],
        'vec14': [0.9999994067108486, 0.9999992993836632, 0.9999996600368343, 0.9999993001910062, 0.9999978161341949],
        'vec15': [0.9999997096418963, 0.9999997482455063, 0.9999996536401479, 0.9999996598468595, 0.4999997337851727],
        'vec16': [0.9999989175114401, 0.9999994540722652, 0.9999998500649301, 0.9999998236360467, 0.9999997152864629],
        'vec17': [0.999998893170208, 0.9999985198582539, 0.9999661689284167, 0.9999963115704804, 0.9999998902791184],
        'vec18': [0.9999997323968299, 0.99999943264924, 0.9999939351338529, 0.9999998841857257, 0.9999752564503278],
        'vec19': [0.9999991683582342, 0.9999995699194852, 0.9999996296366828, 0.999999404634561, 0.999999437952295],
        'vec20': [0.9999986135035015, 0.99999951426488, 0.9999993685953097, 0.9999995920730916, 0.9999995863881384],
        'vec21': [0.9999994643712804, 0.9999998640256966, 0.9999997095727777, 0.9999996548214451, 0.9999995879292917],
        'vec22': [0.9999997303886146, 0.9999994829304617, 0.9999999439581375, 0.9999972741876435, 0.9999992790846255],
        'vec23': [0.9999993012141218, 0.9999993952174684, 0.9999976423569646, 0.4999998998368229, 0.9999996129324576],
        'vec24': [0.9999961177431611, 0.9999967721030967, 0.9999996590819383, 0.9999992158024017, 0.999999571399323],
        'vec25': [0.9999993934268618, 0.9999988139208089, 0.9999974152057078, 0.9999990677925616, 0.9999999238225021],
        'vec26': [0.9999999464233916, 0.9999999430310658, 0.9998849526070603, 0.9999998332801703, 0.9999978763027162],
        'vec27': [0.9999999531513573, 0.9999993060957326, 0.9999998311923398, 0.9999998139468557, 0.9999996984261715],
        'vec28': [0.9998681340825486, 0.999999607072177, 0.9999904067835114, 0.9999999110955234, 0.9999996701762679],
        'vec29': [0.9999992860825326, 0.9999989687035666, 0.9999994209932201, 0.9999990022580708, 0.999999422108353],
        'vec30': [0.9999989479044933, 0.9999999122243397, 0.9999999657356354, 0.9999993243330185, 0.9999999302263995],
        'vec31': [0.9999998009193691, 0.9999994357396365, 0.9999998891922883, 0.9999988564423137, 0.9999999714223263],
        'vec32': [0.9999998107631078, 0.9999990370384069, 0.9999997457735554, 0.9999994342995854, 0.9999999887857132],
        'vec33': [0.9999988037114875, 0.9999985675482113, 0.9999998377125262, 0.9999994487970877, 0.999998937776511],
        'vec34': [0.9999991034150498, 0.9999994912313978, 0.9999997967020682, 0.9999966223452144, 0.9999995426454829],
        'vec35': [0.9999997203025102, 0.999999915286564, 0.9999983075514942, 0.9999996156028648, 0.9999999575054437],
        'vec36': [0.9999999951942575, 0.9999999428233306, 0.9999997069982787, 0.9999999105485687, 0.9999982310310334],
        'vec37': [0.9999995041773511, 0.9999994781688547, 0.9999998720732107, 0.9999999882503863, 0.9999999609955785],
        'vec38': [0.9999994831820865, 0.9999985552184616, 0.9999999435126787, 0.9999997430148806, 0.9999981571822412],
        'vec39': [0.9999999938257396, 0.9999994541761301, 0.9999987057345094, 0.9999996827758548, 0.9999998166673555],
        'vec40': [0.9999988880501689, 0.9999961372054618, 0.9999999778022084, 0.99999999377129, 0.9999992908081524]
    }
}
for num in range(1, 6):
    sub = sorted(os.listdir('./data_322'))[num]
    path = f'./data_322/{sub}'  # path of subfolder
    # print(num, fidelity[f'num{num}'])
    # savemat(f'{path}/fidelity_num{num}_D6_L2.mat', fidelity[f'num{num}'])

In [ ]:
fidelity = {  # D7_L2
    'num1': {
        'vec1': [0.98389995854485, 0.9828675178141613, 0.984036037009809, 0.9817034792347755, 0.9838804694758403],
        'vec2': [0.9817991131462587, 0.9831482129466507, 0.9870912664409381, 0.9844913507453019, 0.9865275114109113],
        'vec3': [0.9994341016150373, 0.9981631475620075, 0.999414489959238, 0.9988776901903407, 0.9923168240128044],
        'vec4': [0.9989866330287644, 0.9997625163594724, 0.9990104178465506, 0.9999765643933968, 0.9999182517594049],
        'vec5': [0.9999984393307516, 0.9999409829815098, 0.9999427296097748, 0.9998251290438629, 0.9999441923069363],
        'vec6': [0.9999888962278236, 0.9999981922036266, 0.999988213216337, 0.9999977938293118, 0.9999710494873951],
        'vec7': [0.9999991014785894, 0.9999942784701797, 0.9999799033511511, 0.9999776028695331, 0.9999839090897391],
        'vec8': [0.9999935454171436, 0.9999994103941899, 0.9999885024414933, 0.9999937864959741, 0.9999957790347981],
        'vec9': [0.9999934017550423, 0.9999972501971738, 0.9999954517710339, 0.9999979578879001, 0.9999975967416928],
        'vec10': [0.9999981381422242, 0.9999963298430039, 0.9999944573013253, 0.9999957931392363, 0.9999963805686547],
        'vec11': [0.9999952952938899, 0.9999989117638198, 0.9999956613138888, 0.9999987915533434, 0.9999988801743166],
        'vec12': [0.9999983641837673, 0.9999977733318378, 0.999997548149498, 0.9999986485486566, 0.9999997025769151],
        'vec13': [0.9999990474132259, 0.999997349540331, 0.9999987546827672, 0.9962282623227338, 0.9999995934634536],
        'vec14': [0.9999997804235641, 0.99999886933983, 0.9999982383951601, 0.9999879947768994, 0.9999980979204366],
        'vec15': [0.9999997653103142, 0.9999989037274583, 0.9999998342262119, 0.9999994471359727, 0.999998832230143],
        'vec16': [0.9999994108477626, 0.9999986684560144, 0.9999986815911046, 0.9999991629321036, 0.9999992201422057],
        'vec17': [0.9999992342829449, 0.9999990079027056, 0.9999994445323657, 0.9999995947944911, 0.9999990490324451],
        'vec18': [0.9999995133270597, 0.9999990964789875, 0.9999985948669001, 0.9999995798498496, 0.9999984352831134],
        'vec19': [0.9999996052668029, 0.9999989158251444, 0.9999988851837576, 0.9999988754874723, 0.9999995273077689],
        'vec20': [0.999999474378372, 0.9999987712536289, 0.999997494723422, 0.9999992194628393, 0.9999993871346068],
        'vec21': [0.9999997095510765, 0.9999975435417199, 0.9999992607215141, 0.9999983728198563, 0.9999999228169367],
        'vec22': [0.9999999043123231, 0.9999995814286905, 0.9999994376851954, 0.9999987584092185, 0.9999997618857471],
        'vec23': [0.9999996089743073, 0.9999988466866014, 0.9999994788940441, 0.9999584434303238, 0.99999894272556],
        'vec24': [0.9999997632398486, 0.9999985737085513, 0.9999958926559172, 0.9999988251415965, 0.9999894177563236],
        'vec25': [0.9999995014383459, 0.9999997504174327, 0.9999997904556395, 0.9999998753902282, 0.9999997538174318],
        'vec26': [0.9999843807623258, 0.9999989905962021, 0.9999998788340387, 0.9999988881419216, 0.9998702284336526],
        'vec27': [0.5000002370349442, 0.9999990521670905, 0.9999987965439416, 0.9999995533316858, 0.999999425627593],
        'vec28': [0.9999990579126383, 0.9999987774628205, 0.9999996193299998, 0.9999997084197052, 0.9999828484495177],
        'vec29': [0.9999915528933679, 0.9999995951855941, 0.9999976428508355, 0.9999991959484196, 0.9999993982448727],
        'vec30': [0.9999993651734973, 0.9999998150713806, 0.9999994745257581, 0.9999906062141508, 0.9999989448789859],
        'vec31': [0.9999988257082251, 0.999999540026141, 0.9999984065757214, 0.9999976768193988, 0.9999979702494357],
        'vec32': [0.9999997233742337, 0.9999991776648113, 0.9999995696123218, 0.9999995610279746, 0.9999999477929713],
        'vec33': [0.9999994164172065, 0.9999985849134186, 0.9999994286651664, 0.9999999447450543, 0.99999941031219],
        'vec34': [0.999999801434044, 0.9999992454954981, 0.9999997427134639, 0.9999983872264642, 0.9999983711658618],
        'vec35': [0.9999982206081797, 0.9999996261961546, 0.9999983477427747, 0.9999948325669497, 0.9999987510116515],
        'vec36': [0.9999997040088136, 0.9999996859497756, 0.999999990928475, 0.9999998958978487, 0.9999996573171199],
        'vec37': [0.9999999764195593, 0.9999982305359522, 0.9999999052496626, 0.9999989273751162, 0.9999999732357413],
        'vec38': [0.9999999094693265, 0.9999997126447565, 0.9999999548544843, 0.9999991497351279, 0.9999997035792285],
        'vec39': [0.9999991223090163, 0.9999959877504001, 0.9999999514435187, 0.9999997416913232, 0.9999997352531396],
        'vec40': [0.9999998875605904, 0.9999998553949135, 0.9999998781918885, 0.9999996163459213, 0.999999920670578]
    },
    'num2': {
        'vec1': [0.9910529783234179, 0.9925075242479772, 0.9908715024938644, 0.9896222931100304, 0.9926992388894417],
        'vec2': [0.988243616018977, 0.98248432242002, 0.9872173588043364, 0.9897425538557527, 0.9861414402977104],
        'vec3': [0.9778445391802666, 0.9891309769055668, 0.9846075972950298, 0.989259151369223, 0.9912503148095265],
        'vec4': [0.9999995928397309, 0.9998528831337001, 0.9997317495243072, 0.999620635424205, 0.9998364365869113],
        'vec5': [0.9999919860521032, 0.999991148841585, 0.9999995400083947, 0.9999900909086878, 0.9999945366516609],
        'vec6': [0.9999965875929011, 0.9999974070905726, 0.999997312047209, 0.9999982678019134, 0.9999966209791208],
        'vec7': [0.9999992011152854, 0.9999975895384473, 0.9999973689059891, 0.9999960806035877, 0.9999990467425319],
        'vec8': [0.9999990450794367, 0.9999993383937277, 0.9999598286769659, 0.999998067345226, 0.9999986957667307],
        'vec9': [0.9999987558582, 0.9999741375236711, 0.9999994958739513, 0.9999990709537566, 0.9999995156492386],
        'vec10': [0.9999996393284488, 0.9999990450851132, 0.9999979890910676, 0.9999993729551231, 0.9999983320272731],
        'vec11': [0.9999988631081499, 0.999999074497407, 0.9999994250807611, 0.9999982099496034, 0.9999996233817624],
        'vec12': [0.9999991437254437, 0.9999992203908505, 0.9999988267747233, 0.9999977846767807, 0.9999993779073971],
        'vec13': [0.9999997344555969, 0.9999992657533541, 0.999999518902072, 0.9999994924204396, 0.9999957421535626],
        'vec14': [0.9999992705337413, 0.9999999158761774, 0.9999998284499975, 0.999999437496135, 0.9999999028873823],
        'vec15': [0.9999992717547207, 0.9999992928577123, 0.9999990461946617, 0.9999993773037824, 0.9999991840895751],
        'vec16': [0.999999801346608, 0.9999992287420819, 0.9999996676413431, 0.9999979244450788, 0.999999220895331],
        'vec17': [0.9999995569955824, 0.9999984272219774, 0.9999994700970942, 0.9999920265155211, 0.9999998236464177],
        'vec18': [0.9999995697289134, 0.9999990481225383, 0.9999991542709677, 0.9999998024137371, 0.9999996683407052],
        'vec19': [0.9999998293070018, 0.999999811007906, 0.999997532423264, 0.9999999461828983, 0.9999998248526782],
        'vec20': [0.9999997625917405, 0.9999999024814451, 0.999999987558113, 0.999999178809784, 0.9999998534183899],
        'vec21': [0.9999995002770987, 0.9999998152121609, 0.999999853366293, 0.999999946448975, 0.9999999203314704],
        'vec22': [0.9999996522944915, 0.9999903385857747, 0.9999983175024282, 0.9999974869606403, 0.999999625028644],
        'vec23': [0.9999989987448711, 0.9999994803907696, 0.999996516002247, 0.9999982990015074, 0.9999996297566048],
        'vec24': [0.9999998542258111, 0.9999954973383715, 0.9999996065602628, 0.9999993651146615, 0.99999936415511],
        'vec25': [0.9999998877123787, 0.9999998508004444, 0.9999999170373701, 0.9999998852332216, 0.9999974355670398],
        'vec26': [0.9999997507025475, 0.9999999363239296, 0.9999996129035786, 0.999999603256628, 0.999999305505635],
        'vec27': [0.9999992371178001, 0.9999981327247908, 0.9999999369334535, 0.9999980506787678, 0.9999996544216292],
        'vec28': [0.9999990885527936, 0.9999999077351373, 0.9999994602636103, 0.9999998994885375, 0.999999902123722],
        'vec29': [0.9999997285279971, 0.9999997396960986, 0.9999998429151922, 0.9999988817255614, 0.9999999509302588],
        'vec30': [0.9999997665203865, 0.9999999741911473, 0.9999992758103315, 0.9999996774060463, 0.9999997143037045],
        'vec31': [0.9999954192850267, 0.9999998035330258, 0.9999989596981107, 0.9999994818832662, 0.9999997804695726],
        'vec32': [0.9999999180015149, 0.9999995718657669, 0.9999995033938978, 0.9999985720356003, 0.9999994614975116],
        'vec33': [0.9999989312829682, 0.9999993973184591, 0.9999987843390756, 0.9999982395095879, 0.9999984946599836],
        'vec34': [0.9999994076681337, 0.9999991261611186, 0.4999998158569038, 0.9999999230985339, 0.9999969858215044],
        'vec35': [0.9999998939315529, 0.9999987065687755, 0.9999994962849895, 0.999999076604613, 0.9999982353408735],
        'vec36': [0.9999999034439803, 0.9999984529745337, 0.999999887727889, 0.9999997918712564, 0.9999999525574146],
        'vec37': [0.9999999239079208, 0.9999998521382513, 0.9999997190472154, 0.9999989522098987, 0.9999993221292042],
        'vec38': [0.9999998737671074, 0.9999992455014712, 0.9999997235654478, 0.9999984953707811, 0.9999989037309034],
        'vec39': [0.9999998110321173, 0.9999995924731627, 0.9999998617121134, 0.9999986069766245, 0.9999999873565678],
        'vec40': [0.9999968441395529, 0.9999989023736044, 0.9999998117168523, 0.9999999078100528, 0.9999998503047107]
    },
    'num3': {
        'vec1': [0.9399240482956567, 0.9288851456075018, 0.9458846587937347, 0.9298394491622713, 0.9341248224504158],
        'vec2': [0.9896876821320962, 0.9898926953722879, 0.9915458225978793, 0.9897635445210139, 0.9898474122731068],
        'vec3': [0.9569575304979507, 0.9630983037455307, 0.9592680805643866, 0.9588925177074973, 0.9536557774907948],
        'vec4': [0.9979797345625149, 0.9933619424520232, 0.9951295897346094, 0.9934569689524826, 0.9926269134237936],
        'vec5': [0.9999707864881427, 0.9998823030278862, 0.9963221455075786, 0.9995504816020707, 0.9998634190901153],
        'vec6': [0.9993687523963092, 0.996679627328778, 0.9998254423564383, 0.9986627748654101, 0.9968716981325672],
        'vec7': [0.9998568329531445, 0.9999440990742188, 0.9999316139531987, 0.9999106381478252, 0.9999183732059751],
        'vec8': [0.999999401958439, 0.9999240196392811, 0.9998841788458395, 0.9999507786522307, 0.9998932248541361],
        'vec9': [0.9999888292549632, 0.9999876846122242, 0.9999946759374745, 0.9999758179382962, 0.999992478008803],
        'vec10': [0.9999891099932833, 0.9999965729037357, 0.9999957965492148, 0.9999970172738439, 0.999994517504843],
        'vec11': [0.9999971335808667, 0.9999974270315517, 0.9999981973319869, 0.9999957206045063, 0.9999931517188225],
        'vec12': [0.9999765608864803, 0.9999960110118737, 0.9999945203309724, 0.9999958067047139, 0.9999999970614369],
        'vec13': [0.9999959294096086, 0.9999979972028092, 0.9999958396809203, 0.999998356449488, 0.9999980054029547],
        'vec14': [0.9999993041703505, 0.9999985130536574, 0.9999972919097525, 0.9999989880138034, 0.9999970572022866],
        'vec15': [0.9999980155833211, 0.9999987315863883, 0.9999985696262986, 0.9999969495465757, 0.9999938826966139],
        'vec16': [0.9999962180368295, 0.9999982060042026, 0.9999978331617918, 0.9999982493100034, 0.9999960671887681],
        'vec17': [0.9999984567438024, 0.999999236696163, 0.9999980611866229, 0.9999989128526607, 0.9999989254284165],
        'vec18': [0.999999204640435, 0.9999720954601441, 0.4999996514708278, 0.9999988572339166, 0.9999982932404341],
        'vec19': [0.9999991310727382, 0.9999988464733416, 0.9999990079742542, 0.9999994444023881, 0.999999071330441],
        'vec20': [0.9999984617152666, 0.9999997533182807, 0.9999992123448244, 0.999999966000598, 0.999996059530794],
        'vec21': [0.999999388218324, 0.9999990390837948, 0.999997181033291, 0.9999992801947425, 0.9999991611486266],
        'vec22': [0.9999977923008051, 0.9999995557830624, 0.9999993972301056, 0.9999998182711411, 0.9999992123905763],
        'vec23': [0.9999958099391273, 0.9999994757012333, 0.9999990234685144, 0.9999993587769478, 0.9999995992718875],
        'vec24': [0.9999994590661596, 0.999999748536081, 0.9999985386265727, 0.9999991111939598, 0.99999886128458],
        'vec25': [0.9999994244059665, 0.9999990446855598, 0.9999995023352231, 0.9999991172323981, 0.9999996162559724],
        'vec26': [0.9999979099792016, 0.9999996092218264, 0.99999953367797, 0.9999998324007797, 0.9999992715481387],
        'vec27': [0.9999983450187377, 0.9999997211553672, 0.9999990761695332, 0.9999996167160629, 0.9999993309607466],
        'vec28': [0.9999996734501914, 0.9999994291405885, 0.9999997492400365, 0.9999991549201156, 0.9999997425164369],
        'vec29': [0.9999990891798841, 0.9999996828841751, 0.9999994416686965, 0.9999996632333583, 0.9999998437339966],
        'vec30': [0.9999993956154292, 0.999999708721116, 0.9999999711527296, 0.9999991021008421, 0.9999996930622393],
        'vec31': [0.9999992854464997, 0.9999996471961601, 0.9999996137902394, 0.999999571088718, 0.9999982682995883],
        'vec32': [0.9999996514122214, 0.999999869672045, 0.9999993521291785, 0.9999994407907056, 0.9999981865243881],
        'vec33': [0.9999996551900491, 0.9999988694898949, 0.9999996776852622, 0.9999996530427905, 0.9999995866883349],
        'vec34': [0.999999633477056, 0.999999938195075, 0.9997311780626092, 0.9999996160756526, 0.999999759048823],
        'vec35': [0.9999994292200326, 0.9999993980421451, 0.9999982287055835, 0.9999996893236772, 0.9999995195077024],
        'vec36': [0.9999996960671707, 0.9999994875862115, 0.9999860583004365, 0.9999986799496834, 0.9999989310500597],
        'vec37': [0.874999359304423, 0.9999997838137765, 0.999999926624364, 0.9999977048846338, 0.9999999905114987],
        'vec38': [0.9999992101052786, 0.9999989948769439, 0.9999978152670616, 0.9999997231084926, 0.999999583140553],
        'vec39': [0.9999994899203488, 0.9999996972581622, 0.9999890927960099, 0.9999982650177357, 0.999999614009619],
        'vec40': [0.9999998484265014, 0.9999996501623065, 0.9999998874165426, 0.9999992374820986, 0.999999777636577]
    },
    'num4': {
        'vec1': [0.9589834349261468, 0.9588594205144093, 0.9386837461308295, 0.9517109995086955, 0.9645677173521617],
        'vec2': [0.9735459922241713, 0.9761986177034401, 0.9778820808480676, 0.976872779612517, 0.9764760115422164],
        'vec3': [0.9740249322062041, 0.9759501641236143, 0.9779308550224338, 0.9748993116575964, 0.9758900456868014],
        'vec4': [0.9111158505306096, 0.9823778092120423, 0.9843987249595771, 0.98423533263, 0.9860306955372988],
        'vec5': [0.9995309643120206, 0.9999409002246633, 0.9420380723579621, 0.9999135667425078, 0.9997709927185174],
        'vec6': [0.9983869878058538, 0.9952263021255814, 0.9957703234518392, 0.9981456524809945, 0.9983676220070437],
        'vec7': [0.9998834043586855, 0.9997029770868998, 0.9998582517841517, 0.9999897203425765, 0.9997484930263796],
        'vec8': [0.9999883517840743, 0.9999122215826896, 0.9999985303162459, 0.9999349960945837, 0.9990701361864537],
        'vec9': [0.9999125783372821, 0.9999032670336245, 0.9999138584719384, 0.9999043144775102, 0.9998727196994364],
        'vec10': [0.9999789043423227, 0.9999924423529873, 0.9999910537358836, 0.9999813349474038, 0.9999871823107966],
        'vec11': [0.9999885791764064, 0.9999843038760061, 0.9999838246933346, 0.9999927166621256, 0.9999966157705709],
        'vec12': [0.9999909105627081, 0.9999916089813804, 0.9999922253902016, 0.9999966369411899, 0.9999916476642454],
        'vec13': [0.9999910384524696, 0.9999950438907412, 0.9999933223823068, 0.99999637778539, 0.9999963684015515],
        'vec14': [0.999999657347126, 0.9999980303647638, 0.9999970215529808, 0.9999945895561871, 0.999999686687591],
        'vec15': [0.9999958285483191, 0.9999955929266344, 0.9999951017606359, 0.9999985848454878, 0.9999946427301027],
        'vec16': [0.9999972201385406, 0.9999960239292638, 0.9999971586008986, 0.9999997938117815, 0.9999984458266815],
        'vec17': [0.9999974426089793, 0.999999416889483, 0.954126767231133, 0.9999979142504685, 0.9999970082128844],
        'vec18': [0.9999985491268056, 0.9999982033982417, 0.9999990232216578, 0.9999974396141365, 0.9999984030921651],
        'vec19': [0.9999982307177597, 0.9999980138293499, 0.9999979062111156, 0.9999991471374687, 0.999998242293852],
        'vec20': [0.9999999370402256, 0.999997322778087, 0.9999985656662039, 0.999999307100235, 0.9999982224310369],
        'vec21': [0.9999984606685214, 0.9999983173904426, 0.9999982114899875, 0.9999986864880516, 0.9999987572124235],
        'vec22': [0.9963448264708323, 0.99999813855349, 0.9999864230999248, 0.9999903544554206, 0.9999984312921236],
        'vec23': [0.9999995698295242, 0.9999993488508114, 0.999999049262301, 0.9999991664819813, 0.9999984572727741],
        'vec24': [0.9999977097850915, 0.9999984352815445, 0.9999991760378414, 0.9999983335115888, 0.9999997471298524],
        'vec25': [0.9999987011834799, 0.9999987512341129, 0.9999992249027922, 0.9999988803821929, 0.9999991349060997],
        'vec26': [0.9999972501335734, 0.9999991433236184, 0.9999995716883512, 0.9999997396355015, 0.9999992200546167],
        'vec27': [0.9999993890403381, 0.9999992030056658, 0.9999994119170635, 0.9999992754980918, 0.9999996768750294],
        'vec28': [0.9999991293817988, 0.9999994061178173, 0.9999992394958048, 0.9999991924545266, 0.9999992828714531],
        'vec29': [0.999999158430595, 0.9999995066693856, 0.9999990713438138, 0.9999994924612363, 0.9999998251655122],
        'vec30': [0.9999992023034, 0.9999999970259205, 0.9999992303073183, 0.9999993617669687, 0.9999979374804718],
        'vec31': [0.999998922059052, 0.9999951828758628, 0.99999947604693, 0.9999994429863684, 0.9999995722859236],
        'vec32': [0.9999992999336744, 0.9999994079536683, 0.9999994740738424, 0.9999917559113614, 0.9999995444342836],
        'vec33': [0.999999311657015, 0.9999994827165287, 0.9999991568602655, 0.9999998341128153, 0.9999995635557717],
        'vec34': [0.9999998951389018, 0.9999991139183143, 0.9999992090355957, 0.9999991773244746, 0.9999971510904058],
        'vec35': [0.9999992387453037, 0.9999996108006359, 0.9999997424488652, 0.9999994475491164, 0.9999994157662273],
        'vec36': [0.9999995922706508, 0.9999995273817097, 0.9999995050969545, 0.9999995609382685, 0.9999998803306955],
        'vec37': [0.9999995896363824, 0.9999990892056265, 0.999997797986208, 0.9999981578681055, 0.9999999149860918],
        'vec38': [0.9999990873133078, 0.9999992675558788, 0.9999994680367245, 0.9999996125591435, 0.9999994461615108],
        'vec39': [0.9999996072161158, 0.9999994674475284, 0.9999998715394909, 0.9999995926708141, 0.9999995838351775],
        'vec40': [0.9999997998311044, 0.999998864856436, 0.9999983341532973, 0.9999995607191157, 0.9999990369294866]
    },
    'num5': {
        'vec1': [0.9911915547262982, 0.9921442585456779, 0.9920295082386614, 0.9913011703094949, 0.9927813701610596],
        'vec2': [0.9805539542596449, 0.9711170959273385, 0.9652850181090271, 0.9482159507358854, 0.973098161934814],
        'vec3': [0.9935507969379217, 0.9930183802211592, 0.9950206627579088, 0.9923536335005035, 0.9923230306774631],
        'vec4': [0.9931374622340542, 0.9963697742424574, 0.994632490603856, 0.997250006448678, 0.9960738508016903],
        'vec5': [0.9976075281805483, 0.9985662125113216, 0.9982028605703467, 0.9992524915119195, 0.9984166748479624],
        'vec6': [0.9999783816210829, 0.9999671192070453, 0.9999717487081001, 0.99996028686602, 0.9999673387683403],
        'vec7': [0.9999823630661039, 0.999981605598856, 0.9999819244564776, 0.99998488723217, 0.9999776313535744],
        'vec8': [0.9999904844386938, 0.9999908681828713, 0.4999959048734548, 0.9999959186477703, 0.9999922997650643],
        'vec9': [0.9999814751967533, 0.9999939057705869, 0.9999947035270234, 0.9999904606166174, 0.9999927085572158],
        'vec10': [0.9999963142471457, 0.9999972454177198, 0.999996411172579, 0.99999864956341, 0.9999958808456234],
        'vec11': [0.9999984643562065, 0.9999979814354863, 0.9994360318487018, 0.9999967165990009, 0.9999966757884012],
        'vec12': [0.9999977382790374, 0.999998334606177, 0.9999984104629198, 0.9999977086537393, 0.9999983846120457],
        'vec13': [0.9999986542725855, 0.9999983637188148, 0.999998781536586, 0.9999989030214256, 0.9999987243499597],
        'vec14': [0.9999991493753518, 0.9999991107859282, 0.99999859436451, 0.9999990087296153, 0.9999992643852806],
        'vec15': [0.9999993539612546, 0.9999986925149749, 0.9999978664283597, 0.9999986585298758, 0.9999990072911461],
        'vec16': [0.9999989102589508, 0.9999994568788391, 0.9999995419136629, 0.9999947836742918, 0.9999984639190018],
        'vec17': [0.9999995479900824, 0.9999994641694263, 0.9999990889346256, 0.9999994559282965, 0.9999994996245622],
        'vec18': [0.9999998123316028, 0.4999996985349876, 0.9999995957350328, 0.999998829688846, 0.9999996763506478],
        'vec19': [0.9999993473594407, 0.9999989997297107, 0.9999996695345825, 0.9999999150452528, 0.9999992457320511],
        'vec20': [0.9999998003349979, 0.9999997194635435, 0.9999998163608389, 0.9999990895024546, 0.9999997645929625],
        'vec21': [0.999999848490123, 0.9999983446781144, 0.9999998970980141, 0.9999997726732783, 0.999999871689809],
        'vec22': [0.999999896020901, 0.9999998291839267, 0.999999504395446, 0.9999993659607616, 0.999999298422649],
        'vec23': [0.9999987856721025, 0.7490910973951511, 0.9999997997698651, 0.9999998600506945, 0.9999992577522459],
        'vec24': [0.9999987644335462, 0.9999987720027341, 0.9999990072568051, 0.9999997502821643, 0.9999997651924439],
        'vec25': [0.9999996901466173, 0.9999974475145507, 0.9999795135642455, 0.9999999057701796, 0.9999997728908911],
        'vec26': [0.9999998943578818, 0.9999998459888075, 0.9999998316764145, 0.9999991841513438, 0.9999995039350364],
        'vec27': [0.9999992572830321, 0.999999836345168, 0.9999998476429743, 0.9999994130164883, 0.9999999585097006],
        'vec28': [0.9999975213167007, 0.9999999722704902, 0.999999642331017, 0.9999935628961791, 0.9999986555399357],
        'vec29': [0.9175832811511889, 0.9999960681501601, 0.9999998170331577, 0.9999996677257641, 0.9999994224490434],
        'vec30': [0.9999997853458155, 0.9999999203074299, 0.9999999203083142, 0.9999998121304678, 0.9999998989898353],
        'vec31': [0.9999994528261451, 0.49999998166949877, 0.9999999111459039, 0.9999998979229544, 0.999998951185777],
        'vec32': [0.9999993086431199, 0.9999995433632177, 0.9999999786169521, 0.9999993334084617, 0.999993517773377],
        'vec33': [0.9999994680384232, 0.9999997276214573, 0.9999994186285159, 0.9999986964746985, 0.9999957522543318],
        'vec34': [0.9999999663738224, 0.9999991584242558, 0.9999996654645754, 0.9999993727928224, 0.9999897234218419],
        'vec35': [0.9999999804889957, 0.9999987909672845, 0.9999215057899512, 0.9999994190410504, 0.9999987050582543],
        'vec36': [0.9999984365666968, 0.9999995799266495, 0.9999997262490471, 0.9999998626891218, 0.9999999532928925],
        'vec37': [0.9999999584909167, 0.9999998487407856, 0.9999964702274404, 0.9999996604374272, 0.9999993594084172],
        'vec38': [0.999997868146879, 0.9999989522953773, 0.9999990671288441, 0.9999999841194407, 0.9999989208777924],
        'vec39': [0.9999985631777141, 0.9999977816862928, 0.9999995352135036, 0.9999997089849464, 0.9999988901671997],
        'vec40': [0.9999997335943753, 0.9999987435884935, 0.9999994954295228, 0.999998138579806, 0.9999997379365977]
    }
}
for num in range(1, 6):
    sub = sorted(os.listdir('./data_322'))[num]
    path = f'./data_322/{sub}'  # path of subfolder
    # print(num, fidelity[f'num{num}'])
    # savemat(f'{path}/fidelity_num{num}_D7_L2.mat', fidelity[f'num{num}'])

In [ ]:
fidelity = {  # D8_L2
    'num1': {
        'vec1': [0.972543797114964, 0.9735255636521295, 0.9781441288757476, 0.9704163066452806, 0.9695736852636702],
        'vec2': [0.9826713612583715, 0.9857574761538127, 0.9803356796253114, 0.9844601463861977, 0.9818715004244348],
        'vec3': [0.9916228741310072, 0.9920702902202961, 0.989367449852135, 0.9849742747893325, 0.9928807665855799],
        'vec4': [0.9994443387950828, 0.999570919229392, 0.9994308284102538, 0.9922987011158158, 0.9976171167386506],
        'vec5': [0.999965198733016, 0.9992888344381807, 0.9999780989323358, 0.999437940834633, 0.9995568114033093],
        'vec6': [0.7501597550507122, 0.9999996212525999, 0.9999905429469468, 0.9995626197985209, 0.9999151721329882],
        'vec7': [0.9999910745204789, 0.9999709677439166, 0.9999795710228419, 0.9999787435762644, 0.9999825623360596],
        'vec8': [0.9999997771027647, 0.9999955372316426, 0.9999975103534896, 0.9999947285866448, 0.9999951610056235],
        'vec9': [0.9999969435704108, 0.9999941427667194, 0.9999931650519268, 0.9999940770312378, 0.9999930927280176],
        'vec10': [0.9999964540526483, 0.9999957251033916, 0.9999982089795514, 0.9999956798384946, 0.99999596636917],
        'vec11': [0.999995306311807, 0.9999990842400361, 0.9999984446173298, 0.999996855980366, 0.9999969107860905],
        'vec12': [0.9999936259982348, 0.9999993153417555, 0.9999969483438869, 0.9999982568164655, 0.9999980799073115],
        'vec13': [0.9999997064153227, 0.9999984709472847, 0.9999981741832118, 0.9999967084753827, 0.9999971179596954],
        'vec14': [0.999999037621957, 0.9999988534869344, 0.9999993473597183, 0.9999984751731161, 0.9999995981765399],
        'vec15': [0.999998647068602, 0.999998971441148, 0.999999191334235, 0.9999874631513677, 0.9999990117083084],
        'vec16': [0.9999991369409702, 0.9999991251247193, 0.999999544426555, 0.9999993899888713, 0.9999998788456294],
        'vec17': [0.9999992039720731, 0.9999995689601102, 0.9999989587018426, 0.9999990613741812, 0.9999987991371656],
        'vec18': [0.9999997056478054, 0.9999991670304975, 0.999999307602661, 0.9999994745966075, 0.9999935794802726],
        'vec19': [0.9999949997750309, 0.9999995981600499, 0.9999982901337867, 0.9999995927305968, 0.999998793605082],
        'vec20': [0.999999847939155, 0.999997995624789, 0.9999994852284071, 0.9999997077485812, 0.9996368483442287],
        'vec21': [0.9999998897901986, 0.9999965872436366, 0.9999843468199859, 0.9999981972040273, 0.999999777542208],
        'vec22': [0.9999999335951448, 0.9999996964344438, 0.9999993290487361, 0.9999995921002732, 0.999997936745919],
        'vec23': [0.9999992906977767, 0.9999990015285031, 0.999999781540216, 0.9999994824971377, 0.99999949946123],
        'vec24': [0.9999980482918587, 0.9999997057745477, 0.9999984132714843, 0.9999996770809052, 0.9999997956909513],
        'vec25': [0.9999996670954975, 0.9999991540577691, 0.9999995847132818, 0.9999998291621607, 0.9999998612014803],
        'vec26': [0.9999991984430914, 0.9999998493596928, 0.9999981595132578, 0.9999995321609731, 0.9999951760017594],
        'vec27': [0.9999998544811196, 0.9999992208078639, 0.9999958096156949, 0.9999976632049342, 0.9999998511091666],
        'vec28': [0.9999998632338787, 0.9999998182987123, 0.9999987264850952, 0.9999984103298966, 0.9999994957382858],
        'vec29': [0.9999994352516117, 0.9999978521416568, 0.9999999409963312, 0.9999997006560332, 0.9999998867175627],
        'vec30': [0.9999989582916778, 0.9999997006485123, 0.9999994087408681, 0.999999870750466, 0.9999998288854784],
        'vec31': [0.999999753555942, 0.9999987834886648, 0.9999998530021904, 0.999998939204138, 0.9999998299813269],
        'vec32': [0.9999998865452304, 0.9999986861176944, 0.9999995040665997, 0.9999997812949025, 0.9999980860273157],
        'vec33': [0.9999990313436873, 0.9999994485464723, 0.9999998508113657, 0.9999986912822021, 0.9999998680637882],
        'vec34': [0.9999974671826208, 0.999999955154468, 0.9999988451390511, 0.9999982893407677, 0.9999999524064264],
        'vec35': [0.999999795216229, 0.9999993535460207, 0.9999995860482613, 0.9999991403601362, 0.9999998205501971],
        'vec36': [0.9999997953581212, 0.9999981505430409, 0.9999997929125146, 0.9999999245180684, 0.9999974657179749],
        'vec37': [0.9999998940449797, 0.9999991990823924, 0.9999999130304958, 0.9999994964988145, 0.9999901363857384],
        'vec38': [0.9999998363828693, 0.9999994063672507, 0.9999999105277012, 0.9999995854960794, 0.9999994428580105],
        'vec39': [0.9999984082489213, 0.9999993655111801, 0.9999995540560371, 0.9999998966601161, 0.9999972501925011],
        'vec40': [0.9999988500765506, 0.9999997422579826, 0.9999982805839552, 0.9999999097465883, 0.9999955239194114]
    },
    'num2': {
        'vec1': [0.9920517565737782, 0.9975956755964125, 0.9948501836021375, 0.9953503956513662, 0.9941039403408709],
        'vec2': [0.9644815326025485, 0.9582026847044973, 0.9591037054788345, 0.9555787883911366, 0.9519056055127366],
        'vec3': [0.9382838385526365, 0.9500363904457737, 0.9559709544594819, 0.9723569533121629, 0.9621561492445752],
        'vec4': [0.9997922293670024, 0.9998294130103668, 0.9996149612752192, 0.999952721200611, 0.999811700281568],
        'vec5': [0.9999901779183528, 0.999990617117623, 0.9999997746271292, 0.9999956189329392, 0.9999972154185413],
        'vec6': [0.9999939913059511, 0.9999991550657739, 0.9999966417001939, 0.9999852859948668, 0.9999962395926156],
        'vec7': [0.9999988716210201, 0.9999976578278984, 0.9999995027770621, 0.9999977881538595, 0.9999962074774132],
        'vec8': [0.99999959243684, 0.9999984664929802, 0.9999975121288165, 0.9999976170136075, 0.9999983949302733],
        'vec9': [0.9999981513694504, 0.9999998207932066, 0.9999992150263454, 0.9999990612335523, 0.9999984211729358],
        'vec10': [0.9999976807325328, 0.9999997583280147, 0.999999432086259, 0.999999354690775, 0.9999993626081899],
        'vec11': [0.9999995812209188, 0.9999995549217845, 0.9999994894209113, 0.9999960338212518, 0.9999995123292517],
        'vec12': [0.9999998144990652, 0.9999991210762911, 0.9999994449950824, 0.9999994765782431, 0.999999554420266],
        'vec13': [0.9999994590142087, 0.9999977203955417, 0.9999992945305466, 0.9999997508948466, 0.9999995471425586],
        'vec14': [0.99999746903078, 0.9999991469397184, 0.999999587439408, 0.9999980556931218, 0.999999576214354],
        'vec15': [0.9999997077466694, 0.9999992766783903, 0.9999990991135731, 0.9999996532680583, 0.9999992715546218],
        'vec16': [0.9999982279714095, 0.9999997195318655, 0.9999996051110615, 0.9999996358057313, 0.9999988519448858],
        'vec17': [0.9999996521034256, 0.9999993452874201, 0.9999984671558092, 0.9999972892255442, 0.9999996904793161],
        'vec18': [0.9999939330042557, 0.9999998159254756, 0.9999994402443364, 0.999998253949975, 0.9999864911768782],
        'vec19': [0.9999989035136955, 0.9999998464043578, 0.9999997486511627, 0.9999975150706146, 0.9999988011711953],
        'vec20': [0.9999967538351292, 0.9999994848568025, 0.9999996193940025, 0.9999987394104468, 0.999999445790665],
        'vec21': [0.9999997207640582, 0.9999999283621045, 0.9999995910629675, 0.9999993705281082, 0.9999996197219461],
        'vec22': [0.9999994212613428, 0.9999998615427588, 0.9999993842384693, 0.9999996909676048, 0.99999956022694],
        'vec23': [0.9999992503433287, 0.9999997366713105, 0.9999997091715581, 0.9999994432830971, 0.9999995709238063],
        'vec24': [0.999997063807085, 0.9999998210810915, 0.9999992744598409, 0.9999991662673658, 0.9999999111991786],
        'vec25': [0.9999993572849583, 0.999998776645267, 0.9999937220070625, 0.9999997497401559, 0.9999996477083333],
        'vec26': [0.9999998681401495, 0.9999997743119706, 0.999999795953395, 0.9999996490303834, 0.9999998433600583],
        'vec27': [0.9999996185812692, 0.9999985472499342, 0.9999998656623186, 0.99999957963662, 0.9999996996490581],
        'vec28': [0.9999987841138535, 0.9999998291987534, 0.999999643881076, 0.999999796129735, 0.9999998057273591],
        'vec29': [0.9999989752713421, 0.9999993411083947, 0.9999995238341711, 0.9999995764048393, 0.9999997465460464],
        'vec30': [0.9999999015151746, 0.99871143187369, 0.9999996549104438, 0.9999996875756922, 0.9999999627338029],
        'vec31': [0.9999997065566937, 0.9999989577368227, 0.999999935334297, 0.9999999062182511, 0.9999999042762523],
        'vec32': [0.9999983310038727, 0.9999986902545899, 0.9999997202409914, 0.9999998028019762, 0.9999992819592386],
        'vec33': [0.9999999105490759, 0.99999992752113, 0.9999996508836537, 0.9999991855778754, 0.9999989091401674],
        'vec34': [0.999999936022544, 0.9999995831843083, 0.9999992198439974, 0.9999994699573831, 0.9999999158717157],
        'vec35': [0.9999996274773687, 0.9999997943573699, 0.9999999179819535, 0.9999987115821923, 0.9999998950172712],
        'vec36': [0.9999987106863797, 0.9999996343508138, 0.999999309618455, 0.9999991564537783, 0.999999883815931],
        'vec37': [0.9999999985415303, 0.9999993831894692, 0.9999996454737767, 0.999998001571869, 0.9999987624087201],
        'vec38': [0.99998095492173, 0.9999994817805447, 0.9999940403323416, 0.9999996759814144, 0.9999996122582757],
        'vec39': [0.9999999147813446, 0.9999999912226176, 0.999999783779681, 0.9999996007572733, 0.9999995229445786],
        'vec40': [0.9999992545834512, 0.9999702636944615, 0.9999986528582074, 0.9999997366302101, 0.9999967247635526]
    },
    'num3': {
        'vec1': [0.9514495751982279, 0.9530047441678635, 0.9530791230000929, 0.9447744121947701, 0.9461689692722184],
        'vec2': [0.978003805548985, 0.9748901675437258, 0.9796594091056289, 0.9813862904046673, 0.9792784976065514],
        'vec3': [0.977879146540559, 0.9772022088025943, 0.9790136898539396, 0.9762144131466127, 0.9795306952107656],
        'vec4': [0.9996921824328493, 0.9996281990240529, 0.9945743945965841, 0.9997618403979108, 0.9926262826204189],
        'vec5': [0.9999469270160679, 0.9997388279332768, 0.9998660678249275, 0.997307870290634, 0.999472615904703],
        'vec6': [0.9996835614406931, 0.9999150128790062, 0.9997528021661188, 0.999556269052035, 0.9997103530171989],
        'vec7': [0.9997913700242621, 0.99987524734225, 0.9999860674129709, 0.999934048077369, 0.9998722696488862],
        'vec8': [0.99996323895709, 0.999998812693566, 0.9999624600180457, 0.9999629874983404, 0.9999544458351626],
        'vec9': [0.9999957466842879, 0.9999857660423446, 0.9999859602919894, 0.9999857429361543, 0.9999954082352505],
        'vec10': [0.999995783843371, 0.9999949550996378, 0.9999994061248576, 0.9999933119944344, 0.999992067364512],
        'vec11': [0.9999977360913258, 0.9999950637530131, 0.9999989674983576, 0.9999979214491064, 0.9999935896738937],
        'vec12': [0.9999923645275193, 0.9999978502317527, 0.9999946117316111, 0.9999972117856823, 0.9999982772175295],
        'vec13': [0.9999972386393334, 0.9999951874587977, 0.9999842486739272, 0.9999961320351599, 0.5000006714281862],
        'vec14': [0.9999968611834804, 0.9999994812973572, 0.9999998797696648, 0.9999972704208043, 0.9999993468499198],
        'vec15': [0.9999971316018064, 0.9999974534646274, 0.9999964261115905, 0.999987032614223, 0.9999993523704721],
        'vec16': [0.9999605724936873, 0.9999794376845536, 0.9999985333411254, 0.9999976193319423, 0.9999982752816194],
        'vec17': [0.9999984738990195, 0.9999954243487381, 0.9999986475269851, 0.9999989315742033, 0.9999993854490262],
        'vec18': [0.9999991292595798, 0.9999998606190947, 0.9999989609536448, 0.99999876143436, 0.9999993423842449],
        'vec19': [0.9999978815966342, 0.9999982590167118, 0.9999997480154199, 0.999998481062489, 0.9999992475791561],
        'vec20': [0.9999989391617679, 0.9999991658056099, 0.9999989041071327, 0.9999996560760285, 0.9999994939378947],
        'vec21': [0.9999996335124879, 0.9999999571463971, 0.49999990564554486, 0.9999985323279088, 0.999999100784676],
        'vec22': [0.999998393736745, 0.9999996902362185, 0.9999992624333067, 0.9999987640210942, 0.999999538415249],
        'vec23': [0.999999571838404, 0.9999985917514241, 0.9999995148061309, 0.9999984091058548, 0.9999994761129591],
        'vec24': [0.9999995792051023, 0.9999995440479006, 0.9999994277905482, 0.9999999529624309, 0.9999950422805646],
        'vec25': [0.9999992062501659, 0.9999991093248788, 0.9999894925118867, 0.9999981994197096, 0.9999998402633676],
        'vec26': [0.9999993292452468, 0.9999987019500924, 0.9999986652048137, 0.9999912038652895, 0.9999992230464496],
        'vec27': [0.9999993370597088, 0.9999998399744864, 0.9999943561820903, 0.9999985605484952, 0.9999993843251265],
        'vec28': [0.999999380792904, 0.9999990110931438, 0.9999996433630641, 0.9999996851367109, 0.9999996944240992],
        'vec29': [0.9999988074483076, 0.9999991239832717, 0.999999285525931, 0.9999939533472876, 0.9999992933338081],
        'vec30': [0.9999967301934097, 0.9999995223987987, 0.9999980358294667, 0.9999996599598919, 0.9999997345294933],
        'vec31': [0.9999993823430884, 0.9999998773821728, 0.9999997371877887, 0.9999997128527927, 0.999999712704825],
        'vec32': [0.9999998279423898, 0.9999995349583793, 0.9999997649328068, 0.9999996266463486, 0.9999997555922033],
        'vec33': [0.9999993685799635, 0.9999999521181421, 0.9999979065671301, 0.9999997583275851, 0.9999707110716205],
        'vec34': [0.9999997239942292, 0.9999985695609678, 0.9999981463960195, 0.9999987284437899, 0.99999973495286],
        'vec35': [0.9999988329338709, 0.9999994336426835, 0.9999996328800571, 0.9999984504977252, 0.9999996493225023],
        'vec36': [0.9999985784914716, 0.9999989633173525, 0.9999997726478996, 0.9999997804944758, 0.9999997835256413],
        'vec37': [0.9999997161498906, 0.9999995333533176, 0.9999996264063895, 0.9999998161236221, 0.9999992601947161],
        'vec38': [0.9999993118110961, 0.9999941165631897, 0.9999982029515955, 0.9999994954247746, 0.9999988580631347],
        'vec39': [0.9999996286357608, 0.999999654348233, 0.9999995663479921, 0.9999993216024312, 0.999999674621803],
        'vec40': [0.9999996342383553, 0.9999997897181462, 0.9999997516551953, 0.9999997901325663, 0.9999998510758552]
    },
    'num4': {
        'vec1': [0.8973366995912423, 0.8948181710698577, 0.9000065182547127, 0.8954688225104993, 0.9114561191763808],
        'vec2': [0.978781631564874, 0.9760965124506966, 0.9767798698620341, 0.9789589784240683, 0.9789914147690797],
        'vec3': [0.9690852171779724, 0.9727677852391106, 0.9766529737945715, 0.9724823246562102, 0.9695455901014608],
        'vec4': [0.9739549705108135, 0.9716076802533005, 0.9154561971804835, 0.9706131650861395, 0.9707595352674881],
        'vec5': [0.9395811557387225, 0.9583883164287238, 0.9600814045139026, 0.9581729734451792, 0.9902062802960477],
        'vec6': [0.9998995247131716, 0.9998124461315784, 0.9992524787548711, 0.9998769140534505, 0.9999215963593904],
        'vec7': [0.9999050831081726, 0.9999864991341051, 0.9999490255042838, 0.9999005872794458, 0.9999405687187538],
        'vec8': [0.9997730544452277, 0.9999950821594428, 0.9997527221971282, 0.9999899667107522, 0.999850578192061],
        'vec9': [0.9999695694436628, 0.9999658421927049, 0.9999946462131309, 0.9999109446590663, 0.9999610332464316],
        'vec10': [0.9999936121290958, 0.999988086235761, 0.9999932254936891, 0.999993003244756, 0.999977002017821],
        'vec11': [0.9999833577363348, 0.9999946152018069, 0.9999918065157575, 0.9999851812994515, 0.9999854500658203],
        'vec12': [0.9999963955615828, 0.9999933572461274, 0.9999936801922904, 0.9999974859390403, 0.999991169756857],
        'vec13': [0.9999944623263354, 0.9999976998836564, 0.9999920997050441, 0.9999932324030789, 0.9999937258368065],
        'vec14': [0.9999968658650058, 0.9999948477464792, 0.9999944633710302, 0.999994469103926, 0.999993130613806],
        'vec15': [0.9999955051978604, 0.9999984017873447, 0.9999966998013575, 0.9999986120960203, 0.9999954475846162],
        'vec16': [0.999998967686579, 0.9999989907518632, 0.9999965045506916, 0.9999989790120596, 0.999997882618766],
        'vec17': [0.9999970567477536, 0.9999975604970625, 0.9999971287323054, 0.999998856855139, 0.999994742673352],
        'vec18': [0.9999980321078187, 0.9999971225461206, 0.99999924063224, 0.9999975791080552, 0.9999971946278975],
        'vec19': [0.9999993604111677, 0.9999993609412151, 0.9999983805373469, 0.9999994781742324, 0.9999988624407676],
        'vec20': [0.9999982540116541, 0.9999978370813585, 0.9999986900611539, 0.9999982323157207, 0.9999984362654644],
        'vec21': [0.9999999145409615, 0.999998482065027, 0.9999984980067887, 0.9999982508656128, 0.9999995725706537],
        'vec22': [0.9999987878895698, 0.9999986530562491, 0.999999588371411, 0.9999989032618849, 0.9999989639484677],
        'vec23': [0.9999988167348574, 0.9999995309745304, 0.999998607833421, 0.9999989042387669, 0.9999989502507429],
        'vec24': [0.9999995316049666, 0.9999984890894733, 0.9999999011836693, 0.9999999999989646, 0.99999915195836],
        'vec25': [0.9999955705166965, 0.9999992089916321, 0.9999989342123441, 0.9999990071232354, 0.9999999999849478],
        'vec26': [0.9999984902890497, 0.9999991530766631, 0.9999994124671567, 0.9999993940958486, 0.9999991607804588],
        'vec27': [0.9999991644565625, 0.9999986897024169, 0.9999991495448541, 0.9999978254680064, 0.9999989601421438],
        'vec28': [0.999999507073442, 0.999999555692625, 0.9999988884942697, 0.9999990985178802, 0.9999994660955596],
        'vec29': [0.9999993522740658, 0.999999697371807, 0.999999191208224, 0.9999957711557573, 0.9999988699339336],
        'vec30': [0.9999994221128563, 0.9999989767798175, 0.9999991485162738, 0.9999991826467546, 0.9999996802188317],
        'vec31': [0.999998884145983, 0.9999993744334991, 0.9999994585585438, 0.999999836839757, 0.9999995549442724],
        'vec32': [0.999999339720192, 0.9999992298636531, 0.99999994505849, 0.9999977488845319, 0.9999991403459341],
        'vec33': [0.99999925612918, 0.9999996813563584, 0.9999992533859196, 0.9999999993854694, 0.9999995987984459],
        'vec34': [0.9999990143200674, 0.999999220494277, 0.9999977256210657, 0.9999995442180707, 0.9999996082469359],
        'vec35': [0.9999997437325088, 0.9999998780893006, 0.9999998217480951, 0.9999996384909655, 0.9999989102750918],
        'vec36': [0.9999995185801286, 0.9999990757817772, 0.9999994047919359, 0.9999997538584102, 0.9999995380868918],
        'vec37': [0.9999995569806317, 0.9999995402429545, 0.9999988682374331, 0.9999999153911485, 0.9999993454838396],
        'vec38': [0.99999968812839, 0.9999993838142841, 0.9999996002336685, 0.9999996306495269, 0.9999996338175159],
        'vec39': [0.9999986603299407, 0.9999998586105302, 0.9999990695216249, 0.9999996446537142, 0.9999994669409974],
        'vec40': [0.999993418796671, 0.9999998184945759, 0.9999993761074001, 0.9999996240859212, 0.9999997255551406]
    },
    'num5': {
        'vec1': [0.9346659260236017, 0.9312118418510206, 0.9312478152260563, 0.9465136240654184, 0.9400810331370698],
        'vec2': [0.9961980831833667, 0.9954390320168817, 0.9970627122788851, 0.9944638634702393, 0.9946280910775152],
        'vec3': [0.9919040867730784, 0.9913055286247355, 0.9868072128759884, 0.9931604970603434, 0.9914542983681013],
        'vec4': [0.9987092957869419, 0.9984549967009888, 0.9986958264441067, 0.997543908856972, 0.9967532358836623],
        'vec5': [0.9998517158262691, 0.9999028142708822, 0.9998812932251552, 0.9998691119044972, 0.9997596584070909],
        'vec6': [0.9999684358439465, 0.999972422771388, 0.9999710427526785, 0.9999717424030576, 0.999973243496536],
        'vec7': [0.9999884799875222, 0.9999876119372443, 0.999993610400945, 0.9999821923728971, 0.9999862706515242],
        'vec8': [0.9999927169396239, 0.9999904172228018, 0.9999901990005761, 0.9999931764133714, 0.9999907456492776],
        'vec9': [0.9999938974381832, 0.9999930579554156, 0.9999929054376644, 0.9999924453916023, 0.9999954057230975],
        'vec10': [0.9999959965429658, 0.999997580340774, 0.9999967500638215, 0.9999975720067367, 0.9999980344067863],
        'vec11': [0.9999990340421709, 0.999996782703086, 0.9999971419844982, 0.9999942653752537, 0.9999973948131027],
        'vec12': [0.9999964453105442, 0.9999984750037088, 0.9999971638097302, 0.9999973064002321, 0.9999973532109112],
        'vec13': [0.9999816374912494, 0.9999987265948325, 0.9999987186384509, 0.9999987154506405, 0.9999996344347555],
        'vec14': [0.9999992076747539, 0.9999997300925638, 0.9999986848034536, 0.9999992115311005, 0.999998993811785],
        'vec15': [0.9999993774869654, 0.99999804074001, 0.9999995473049387, 0.999999425805284, 0.9999993377108084],
        'vec16': [0.9999993741838715, 0.9999994917429579, 0.9999971994548627, 0.9999994360360365, 0.9999995749333819],
        'vec17': [0.9999994089054641, 0.9999982551575515, 0.9999987150052078, 0.9999996016165916, 0.9999997621506668],
        'vec18': [0.9999990026187273, 0.9999995547735931, 0.9998323665392324, 0.9999990267826397, 0.9999988322463241],
        'vec19': [0.9999994814401733, 0.9999968283186734, 0.9999998072538006, 0.9999976775117791, 0.9999981984169319],
        'vec20': [0.9999992997313498, 0.9999992767037704, 0.9999990427072323, 0.9999996865008598, 0.9999976613642306],
        'vec21': [0.9999997608096768, 0.9999995624736716, 0.9999997206816217, 0.9999994916021039, 0.9999994106125365],
        'vec22': [0.9999998588583184, 0.9999990961036098, 0.9999805605038031, 0.9999994241562118, 0.9997641587879745],
        'vec23': [0.9999955541972803, 0.9999993507768601, 0.9999981754533975, 0.9999994454756307, 0.9999997699233492],
        'vec24': [0.9999999471880626, 0.9999997647143548, 0.9999997239279823, 0.9999997897865148, 0.9999993003432188],
        'vec25': [0.9999997044794802, 0.9999997102743655, 0.9999998410763371, 0.9999993140182806, 0.999999629379425],
        'vec26': [0.99999968076725, 0.9999998256898206, 0.9999987564468908, 0.9999992767643009, 0.9999991176925795],
        'vec27': [0.9999996643534519, 0.9999818207574779, 0.9999999102882249, 0.9999999336991225, 0.9999987701464156],
        'vec28': [0.9999989964184035, 0.9999996285055285, 0.9999982093187522, 0.9999997425520227, 0.999998618486089],
        'vec29': [0.9999990066561104, 0.9999997209875298, 0.9999997481960597, 0.9999981331574199, 0.9999997584483064],
        'vec30': [0.9999999771228618, 0.9999996877723539, 0.999999437403401, 0.9999999378832647, 0.9999998588980494],
        'vec31': [0.9999995032124154, 0.9999999708027204, 0.9999999123199591, 0.9999990513098846, 0.9999999312540077],
        'vec32': [0.9999994832532663, 0.999995817896963, 0.9999991172931243, 0.9999995249867137, 0.9999995147339013],
        'vec33': [0.9999999826528447, 0.9999997383109749, 0.9999999206360926, 0.9999971493767519, 0.9999998499397392],
        'vec34': [0.9999990526262599, 0.9999998979205849, 0.9999997907241315, 0.9999993543913427, 0.9999985649111045],
        'vec35': [0.9999965358354237, 0.9999987978507425, 0.9999999812629864, 0.9999971442740526, 0.9999960655075599],
        'vec36': [0.9999975997553983, 0.9999994951704108, 0.9999995693738811, 0.9999988652171456, 0.9999992274833234],
        'vec37': [0.9999989464851281, 0.9999996535711572, 0.999999717552009, 0.9999995839191669, 0.9999992752109224],
        'vec38': [0.9999994762034112, 0.999998205848142, 0.9999999624491903, 0.9999993742342819, 0.9999998871335729],
        'vec39': [0.9999993674510093, 0.9999999892264464, 0.9999998541324561, 0.9999994116486479, 0.9999977413294603],
        'vec40': [0.9999999875205264, 0.9999998861969818, 0.9999993602134238, 0.9999989513976648, 0.9999997577817465]
    }
}
for num in range(1, 6):
    sub = sorted(os.listdir('./data_322'))[num]
    path = f'./data_322/{sub}'  # path of subfolder
    # print(num, fidelity[f'num{num}'])
    # savemat(f'{path}/fidelity_num{num}_D8_L2.mat', fidelity[f'num{num}'])

In [ ]:
fidelity = {  # D9_L2
    'num1': {},
    'num2': {},
    'num3': {},
    'num4': {},
    'num5': {}
}
for num in range(1, 6):
    sub = sorted(os.listdir('./data_322'))[num]
    path = f'./data_322/{sub}'  # path of subfolder
    # print(num, fidelity[f'{num}'])
    # savemat(f'{path}/fidelity_num{num}_D8_L2.mat', fidelity[f'num{num}'])

In [ ]:
# 2024-01-23 D5_L2, Eval: 500, loss_tol: 1e-12, gtol: 1e-12
fidelity = {
    'num1': [
        0.9993232272250057, 0.9914233490137979, 0.9965275014354146, 0.9996325521405218, 0.9998639807958072,
        0.9999992977900146, 0.9999856779830664, 0.9999991904503778, 0.9999958836065987, 0.999995640882225,
        0.9999987004445674, 0.9999982417782245, 0.9999983957084034, 0.999998486753758, 0.9999981250556182,
        0.9999990499578681, 0.9999969995743045, 0.9999986244254488, 0.9999954726445964, 0.999999825233874,
        0.9999997980223443, 0.9999997736437963, 0.9999980759818685, 0.9999996005218132, 0.9999995322587378,
        0.9999985266754758, 0.999998906884545, 0.9999998139887376, 0.9999989829769778, 0.9999983390995074,
        0.9999998378080778, 0.9999998525582914, 0.9999998492670878, 0.9999999278672919, 0.9999984872807368,
        0.9999997778063506, 0.9999993771382574, 0.999999808592916, 0.9999979754337459, 0.9999997414899132
    ],
    'num2': [
        0.9979662582983877, 0.9986813141225841, 0.9986110217279236, 0.9999806470578718, 0.9999993228547944,
        0.9999992415778446, 0.9999921929216568, 0.999998603426064, 0.9999995133830337, 0.9999957179183333,
        0.9999995404837912, 0.9999994744958542, 0.999997973836509, 0.9999992916360538, 0.9999999551282245,
        0.9999989710623822, 0.999999294243354, 0.9999999065056036, 0.9999981730856145, 0.9999996992857818,
        0.9999994143178819, 0.9999994348665251, 0.9999997618542881, 0.9999994126157228, 0.9999998299172084,
        0.9999979156803651, 0.9999994491719608, 0.9999993727741054, 0.9999992732759453, 0.9999991676297215,
        0.9999975699933445, 0.9999997191176849, 0.9999998518987606, 0.9999995304567821, 0.9999998581641676,
        0.9999991900070252, 0.9999996223723426, 0.9999985356053407, 0.9999993164926276, 0.9999990595303969
    ],
    'num3': [
        0.997884142960926, 0.9962932798309575, 0.9978683463354528, 0.998692421144387, 0.9998828209948645,
        0.9999997119376925, 0.9999934012525176, 0.9999992354298608, 0.9999869715282645, 0.9999960739471114,
        0.9999932346812268, 0.9999981844436938, 0.9999963044368534, 0.9999975837759773, 0.9999980351882184,
        0.9999982587787936, 0.9999986818891444, 0.9999995732835749, 0.9999990904208054, 0.9999998615322909,
        0.9999991624649848, 0.9999979772398605, 0.9999995562481583, 0.9999993585034808, 0.9999998699562168,
        0.9999982941912672, 0.9999990704496844, 0.9999994424663888, 0.9999995843422083, 0.9999994647366552,
        0.9999993941905159, 0.9999997446789913, 0.999999545573097, 0.9999986688365781, 0.9999998282743686,
        0.9999996714168332, 0.999996555505988, 0.9999998188667629, 0.9999975764125631, 0.9999996769884644
    ],
    'num4': [
        0.9953136669436368, 0.9859819639706634, 0.9978742825680968, 0.9986863683900956, 0.9999112029727695,
        0.9996019005545698, 0.9999693099343747, 0.9999512585440459, 0.9999900935458298, 0.9999967168505779,
        0.9999827281290624, 0.9999907526197065, 0.9999958567866161, 0.9999979704657429, 0.9999986934533525,
        0.9999981862902961, 0.9999968625340763, 0.999999368838002, 0.9999993427865178, 0.9999995474394994,
        0.9999987546625542, 0.999998784455929, 0.9999982998950186, 0.9999987494324905, 0.9999990983425439,
        0.9999989157915083, 0.9999989522964973, 0.9999991283048941, 0.9999983336809564, 0.9999993219169212,
        0.999999145936841, 0.9999996783062486, 0.9999999088945571, 0.9999987761661127, 0.9999993427066931,
        0.999999744449307, 0.9999998681771816, 0.9999994744734702, 0.9999993896152823, 0.9999996840631682
    ],
    'num5': [
        0.998094987102364, 0.997150498211435, 0.9997021296009085, 0.9999350393023094, 0.9999789826179868,
        0.9999927450457824, 0.9999935207991452, 0.9999926585435045, 0.9999983887131371, 0.9999968929237789,
        0.999999047764252, 0.9999991540823437, 0.9999966693492639, 0.9999995863283111, 0.9999985419235853,
        0.999999742523371, 0.9999995009210385, 0.9999971453926064, 0.9999997670609501, 0.999999533637724,
        0.9999997548181669, 0.9999999469847443, 0.9999999447302454, 0.9999994380233928, 0.9999985536382625,
        0.9999989074045436, 0.9999987541429545, 0.9999999205354158, 0.9999997952981681, 0.9999999427363444,
        0.9999997954732425, 0.9999999650250315, 0.9999999628916534, 0.9999999440169342, 0.9999996773429773,
        0.9999988773835712, 0.9999996706767157, 0.9999996051057174, 0.9999989816055794, 0.9999998420662505
    ]
}
for num in range(1, 6):
    sub = sorted(os.listdir('./data_322'))[num]
    path = f'./data_322/{sub}'  # path of subfolder
    print(num, fidelity[f'num{num}'])
    # savemat(f'{path}/fidelity_num{num}_D5_L2.mat', {'fidelity': fidelity[f'num{num}']})

In [ ]:
# 2024-01-19 D9_L2, Eval: 500, loss_tol: 1e-12, gtol: 1e-12
fidelity = {
    'num1': [
        0.9617554833246672, 0.9704350710275164, 0.9892862158667609, 0.9994392735740232, 0.9998335210796601,
        0.9999791735702322, 0.9999788854135511, 0.999989156906408, 0.9999989165174641, 0.9999957803995483,
        0.9999971372589765, 0.999996203411881, 0.9999985297680456, 0.9999984629070369, 0.9999988485035154,
        0.9999974062835336, 0.9999969194961101, 0.9999986975735274, 0.9999988170750815, 0.9999989719844034,
        0.9999990104454513, 0.9999997399393973, 0.9999995960243385, 0.9999997451710452, 0.9999994909393108,
        0.9999996450021515, 0.9999989873463905, 0.9999996344985846, 0.9999988209181478, 0.9999994453651209,
        0.9999984279882854, 0.9999996496724832, 0.9999958650163461, 0.99999991002494, 0.9999997620559244,
        0.99999981375596, 0.9999996697206308, 0.9999998698779831, 0.9999998632544055, 0.9999997800655305
    ],
    'num2': [
        0.9979403341674667, 0.9987410136200778, 0.9981149476240618, 0.9999645534801196, 0.9999893465239353,
        0.9999939646096054, 0.9999981041119880, 0.9999962981831538, 0.9999985117846281, 0.9999997846801302,
        0.9999997334590828, 0.9999992245043285, 0.9999997166872224, 0.9999995692865434, 0.9999984357145331,
        0.9999986541406416, 0.9999991393414848, 0.9999998425964917, 0.9999983169525943, 0.9999992584916136,
        0.999999311847755, 0.9999996932237183, 0.9999983808207822, 0.9999997597283896, 0.9999998892334339,
        0.9999989094116848, 0.9999994879094852, 0.9999991909399493, 0.9999997739079123, 0.9999999721413593,
        0.9999995861361362, 0.9999999144124769, 0.9999988569907845, 0.9999999026579214, 0.9999989856939172,
        0.9999997555992127, 0.9999995997728244, 0.9999985512281144, 0.99999991880818, 0.9999995185162867
    ],
    'num3': [
        0.933571314471861, 0.9854154582724525, 0.9902265987806878, 0.984422147597338, 0.9990012435708682,
        0.9999039782123605, 0.9999870885996305, 0.9999951446238738, 0.9999895277338212, 0.9999917063648567,
        0.999993638932889, 0.9999974593218525, 0.9999958457901077, 0.9999963908361263, 0.9999981463839414,
        0.9999980480973364, 0.9999986174576252, 0.9999990375845698, 0.9999990121443356, 0.9999989543945097,
        0.9999992071831976, 0.9999979846298215, 0.9999992751079414, 0.9999995477585752, 0.9999991199759194,
        0.9999988757470661, 0.9999997225784405, 0.9999989846950128, 0.999999311349249, 0.9999994580284763,
        0.9999976317526629, 0.9999964224415535, 0.9999994866221147, 0.9999993594134208, 0.9999995336912183,
        0.9999998561078772, 0.9999984693648589, 0.9999998362845882, 0.9999991547225997, 0.9999998956622053
    ],
    'num4': [
        0.8790454029015492, 0.9496673098930248, 0.9859338725834761, 0.9552253609073851, 0.9976386706712298,
        0.9998393021778118, 0.9999687343872679, 0.9999767734284357, 0.9999724404381551, 0.9999901947499402,
        0.9999957401767663, 0.9999908765462867, 0.999991909335785, 0.9999929601087953, 0.9999963553419347,
        0.9999957617994242, 0.9999968634949697, 0.9999991375054063, 0.999997960599073, 0.9999981551994132,
        0.9999986632110596, 0.9999987913169669, 0.9999996934575774, 0.999998862789216, 0.9999991904694544,
        0.9999991489523112, 0.9999999694075506, 0.9999997976463826, 0.9999995453919661, 0.9999994314924819,
        0.999999318684167, 0.9999992417559023, 0.9999989864295674, 0.9999995672318642, 0.9999994475928664,
        0.9999990893645132, 0.9999999640914572, 0.9999995513968798, 0.9999992477510264, 0.9999997575208573
    ],
    'num5': [
        0.9650684920430554, 0.9671393895510254, 0.9959301350667873, 0.9998719060007113, 0.9999458000372964,
        0.999965238977313, 0.9999795956021398, 0.9999926094762437, 0.999994031398969, 0.9999966228979644,
        0.9999968455767154, 0.9999983372056015, 0.999998880500888, 0.9999963787307606, 0.9999993288455189,
        0.9999983647546893, 0.9999992114488288, 0.9999994057269541, 0.9999996611531172, 0.9999996075460148,
        0.9999997577078218, 0.9999998580810578, 0.9999995410066578, 0.999999895580831, 0.9999993659513993,
        0.9999985637395506, 0.9999990954164661, 0.9999997136598899, 0.9999999368942403, 0.9999999637079614,
        0.999999193264875, 0.9999995651364215, 0.9999995769249668, 0.9999997644859813, 0.999995697741791,
        0.9999999454451312, 0.9999994810345582, 0.9999999404875566, 0.9999971438421005, 0.9999994700193074
    ]
}
for num in range(1, 6):
    sub = sorted(os.listdir('./data_322'))[num]
    path = f'./data_322/{sub}'  # path of subfolder
    print(num, fidelity[f'num{num}'])
    # savemat(f'{path}/fidelity_num{num}_D9_L2.mat', {'fidelity': fidelity[f'num{num}']})

In [ ]:
def plot(energy, fidelity):
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.xlim(max(energy) + 0.01, min(energy) - 0.01)
    plt.ylabel('Fidelity', fontsize=12)
    plt.xlabel('Energy', fontsize=12)
    plt.scatter(energy, fidelity)
    plt.plot(energy, fidelity)

    plt.subplot(1, 2, 2)
    fidelity = 1 - fidelity
    plt.xlim(max(energy) + 0.01, min(energy) - 0.01)
    plt.ylim(max(fidelity) * 2, min(fidelity) / 2)
    plt.ylabel('1 - Fidelity', fontsize=12)
    plt.xlabel('Energy', fontsize=12)
    plt.scatter(energy, fidelity)
    plt.plot(energy, fidelity)
    plt.yscale('log')
    plt.show()

In [ ]:
# Save energy list mat
D = 8
for num in range(1, 6):
    sub = sorted(os.listdir('./data_322'))[num]
    path = f'./data_322/{sub}'  # path of subfolder
    key = lambda x: [int(y) if y.isdigit() else y for y in re.split('(\d+)', x)]
    uMPS_name = [i for i in dict_file(f'{path}/uMPS').values() if f'num{num}' and f'D{D}' in i]  # uMPS file name
    uMPS_name = sorted(uMPS_name, key=key)  # sort 1,10,11,...,2 into 1,2,...,10,11
    energy = np.array([loadmat(f'{path}/uMPS/{uMPS_name[i]}')['energy'][0][0] for i in range(len(uMPS_name))])
    # savemat(f'{path}/energy_num{num}_D{D}.mat', {'energy': energy})
    plt.xlim(max(energy) + 0.01, min(energy) - 0.01)
    plt.ylabel('Ergodic', fontsize=12)
    plt.xlabel('Energy', fontsize=12)
    ergodic = np.arange(len(energy))
    plt.scatter(energy, ergodic)
    plt.plot(energy, ergodic)
    plt.show()

In [ ]:
# 2024-01-17 D5_L2, Eval: 1000, loss_tol: 1e-12, gtol: 1e-12
ind, num = 0, 1
path = f'./data_322/old_mat'  # path of subfolder
fidelity = loadmat(f'{path}/fidelity_D5_L2_1000.mat')[f'{num}'][0]
energy = loadmat(f'{path}/energy_D5.mat')[f'{num}'][0]
print('Number of vec:', len(fidelity))
fidelity = fidelity[ind:]
energy = energy[ind:]
plot(energy, fidelity)

In [ ]:
energy_list = {}
for num in range(1, 6):
    path = f'./data_322/old_mat'  # path of subfolder
    key = lambda x: [int(y) if y.isdigit() else y for y in re.split('(\d+)', x)]
    uMPS_name = [i for i in dict_file(f'{path}/uMPS').values() if f'num{num}' in i]  # uMPS file name
    uMPS_name = sorted(uMPS_name, key=key)  # sort 1,10,11,...,2 into 1,2,...,10,11
    uMPS_num = len(uMPS_name)  # number of uMPS in subfolder
    energy = np.array([loadmat(f'{path}/uMPS/{uMPS_name[i]}')['energy'][0][0] for i in range(uMPS_num)])
    energy_list.update({f'{num}': energy})
# savemat(f'{path}/energy_D5.mat', energy_list)
energy_list

In [ ]:
path = f'./data_322/old_mat'  # path of subfolder
fidelity_list = {  # D5_L2_1000
    '1': [
        0.9981289692291885, 0.9996502776244797, 0.9999977325183597, 0.9999999538165608, 0.9999999631621687,
        0.9999999981178249, 0.9999999990491908, 0.9999999870247909, 0.9999999995325286, 0.999999996890498,
        0.9999999996435106, 0.9999999963128423, 0.99999999980957, 0.999999999235185, 0.9999999075029564,
        0.9999999990564923, 0.999999999366403, 0.999999999314469, 0.9999999993636202
    ],
    '2': [
        0.9981389382415938, 0.9999956457715545, 0.9999999989847725, 0.9999999992166471, 0.9999999994119264,
        0.9999999838276614, 0.9999999996881306, 0.9999999977291498, 0.9999999991503257, 0.9999999997033882,
        0.9999999896057856, 0.9999999300315521, 0.9999999998494975, 0.9999999998906166
    ],
    '3': [
        0.9989512240456557, 0.9999786804047124, 0.9999999465087674, 0.9999999798641659, 0.9999999946718774,
        0.9999999972457114, 0.9999999978088169, 0.9999999983712343, 0.9999999979912063, 0.9999999999990524,
        0.9999999983332415, 0.9999999901724372, 0.9999999978404583, 0.9999999999990449, 0.9999999438055598,
        0.9999999997751509, 0.9999999974705225, 0.9999999707900001, 0.9999998734971867, 0.9999999970784021,
        0.999999959234005, 0.9999999977125996, 0.9999999992892423, 0.9999999864215898, 0.9999999978886928,
        0.9999999996152665, 0.9999999854520261
    ],
    '4': [
        0.9656594608333953, 0.9999999488783722, 0.9999999966379637, 0.9999999959912085, 0.9999999993446539,
        0.9999999964659846, 0.9999999948222436, 0.9999999985258305, 0.9999998942063467, 0.9999999988240501,
        0.9999999605588056, 0.9999999783544692, 0.9999999962843825, 0.9999999976736509, 0.9999999724574212,
        0.9999999979586802, 0.9999999976737428, 0.9999999922975977, 0.999999995683543, 0.9999999986328687,
        0.999999996570117, 0.99999999365461, 0.9999999964703045, 0.9999999935444874, 0.9999999968145721,
        0.9999999999990123, 0.9999999299585973, 0.9999999857817421, 0.9999999553321547
    ],
    '5': [
        0.9995150962808503, 0.9999999779451315, 0.9999999734990139, 0.9999999996027182, 0.9999999313142587,
        0.9999999998956567, 0.999999971373524, 0.9999999998434825, 0.9999999359663397, 0.9999999998980577,
        0.9999999993656031, 0.9999999046012363, 0.9999999353031803, 0.9999999999006519, 0.9999999998807851,
        0.9999999998763585, 0.9999999727034927, 0.9999999482890065, 0.9999999997819247, 0.999999873765143,
        0.9999999901369732, 0.9999999989891394
    ]
}
# savemat(f'{path}/fidelity_D5_L2_1000.mat', fidelity_list)

In [ ]:
# 2024-01-17 D5_L2, Eval: 500, loss_tol: 1e-12, gtol: 1e-12
ind, num = 0, 1
path = f'./data_322/old_mat'  # path of subfolder
fidelity = loadmat(f'{path}/fidelity_D5_L2_500.mat')[f'{num}'][0]
energy = loadmat(f'{path}/energy_D5.mat')[f'{num}'][0]
print('Number of vec:', len(fidelity))
fidelity = fidelity[ind:]
energy = energy[ind:]
plot(energy, fidelity)

In [ ]:
path = f'./data_322/old_mat'  # path of subfolder
fidelity_list = {  # D5_L2_500
    '1': [
        0.9963501115273821, 0.9986255378358114, 0.9999968049307384, 0.9999997914850758, 0.9999997630235711,
        0.9999994795736982, 0.9999994315750385, 0.9999996991612291, 0.9999999513689395, 0.999999756578521,
        0.9999997569946619, 0.9999999668131621, 0.9999999571274998, 0.9999994559845724, 0.999999877044508,
        0.9999997666967089, 0.9999994142024324, 0.9999994079751954, 0.9999991333469795
    ],
    '2': [
        0.9936371491921793, 0.9998413895065205, 0.9999999238848767, 0.9999998261529646, 0.9999998730835418,
        0.9999992583367974, 0.9999999490655138, 0.9999999782529083, 0.9999994306438991, 0.99999998229514,
        0.9999995634555819, 0.9999986772332452, 0.9999997568556303, 0.9999992261914328
    ],
    '3': [
        0.9977384054625194, 0.9989687621056889, 0.9999994980624193, 0.9999998579236938, 0.999999554952602,
        0.9999987500964322, 0.9999964864347995, 0.9999982856647585, 0.999999453573, 0.9999995551704526,
        0.9999992718846639, 0.9999999701829814, 0.9999993775394753, 0.9999991911909039, 0.9999998769643839,
        0.9999991063364582, 0.999999593737054, 0.9999974337587729, 0.999997273117633, 0.9999999094956995,
        0.9999944776719654, 0.9999975582651637, 0.9999989867138329, 0.9999999730393971, 0.999999342961981,
        0.9999998004654385, 0.999999997472015
    ],
    '4': [
        0.9765102570471846, 0.9999995635887, 0.9999990165586237, 0.999999992722676, 0.9999996115552525,
        0.9999999939486682, 0.9999994060639792, 0.9999999988290807, 0.9999999081528654, 0.9999999932183138,
        0.9999999968256156, 0.9999998277365552, 0.9999998261919906, 0.9999995425325316, 0.9999999925807567,
        0.999999992429967, 0.9999999511651126, 0.9999990913965662, 0.9999997524760061, 0.9999998121900529,
        0.9999999925896396, 0.9999984536591913, 0.9999990869493365, 0.9999998197242934, 0.9999998796175735,
        0.9999998446402523, 0.999999917076173, 0.9999987717367818, 0.9999998005328963
    ],
    '5': [
        0.9985610759892491, 0.9999990120382192, 0.9999999505590405, 0.9999988868487458, 0.9999977908663584,
        0.9999998749916754, 0.9999988653547986, 0.9999998866431019, 0.9999998604243351, 0.9999999619204065,
        0.9999990184561406, 0.9999996040823064, 0.999999830276298, 0.9999996156967483, 0.9999995599043443,
        0.9999999765103684, 0.9999991302137238, 0.9999862447852008, 0.9999999856409315, 0.9999992354414939,
        0.9999997374931163, 0.9999986284809315
    ]
}
# savemat(f'{path}/fidelity_D5_L2_500.mat', fidelity_list)